# Detección automática de repeticiones parciales en repositorios digitales

El repositorio del proyecto es [Github]().

# TO DO
* generell auf we umschreiben im tutorial-teil

## Why to Read this Tutorial?
* In this tutorial, you will learn to read metadata from an OAI-PMH data provider and how to convert the retrieved data from Dublin Core to a pandas data frame.
* Furthermore, you will carry out some basic data analysis on your data in order to find out if the data is corrupt or unclean. Based on an example, you will clean some aspects of your data.
* Finally, you will analyse and visualize the data with the help of a network graph.

![title](img/architecture.png)

### Preparations
The following code snippet initializes your Python run-time enviroment in order to run all of the subsequent actions.

If you have installed everything correctly with the help of [Anaconda](https://www.continuum.io/downloads) as explained in [dst4l0.ipynb](https://github.com/elektrobohemian/dst4l-copenhagen/blob/master/dst4l0.ipynb), the following packages should be installed correctly and be importable without problems.

In [ ]:
!pip install jellyfish
!pip install geojson
!pip install geopy
#Sickle is a lightweight OAI-PMH client library written in Python. It has been designed for conveniently retrieving data 
#from OAI interfaces the Pythonic way
!pip install sickle
!pip install googlemaps

#### Additional Configuration of This Notebook

In [4]:
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If you set 'allowDownloads' to True, all images will be downloaded and processed again
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
allowDownloads=True # should be True if you run this for the first time

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# Download directory used for images etc.
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
downloadDir="./tmp/"

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# Keep original downloaded TIFF images? Otherwise, they will be deleted and only JPEG thumbnails remain after download.
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

keepTIFFs=False

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If you set 'demoClustering' to False, the clustering steps will take about 2 hours
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
demoClustering=False # should be False if you run this for the first time

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If you set 'allowComputationallyExpensiveCalculations' to True, various time consuming tasks (some of them running 
# several hours) will be carried out and not only loaded from pre-computed data files
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
allowComputationallyExpensiveCalculations=True # should be True if you run this for the first time

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# Enabling this action activates exact string matching againt name lists
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
allowExactStringMatching=True

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If you set to True, some plots will be saved as PDF documents in ./figures/
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
saveFiguresAsPDF=True

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If you set to True, Google Map API will be used to resolve location names, if False OpenStreetMap will be used
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *i
useGoogleMapsAPI=False
privateGoogleKey='ENTER YOUR KEY HERE' #if you want to use Google, you need a personal key

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If set to True, OpenStreetMap API will be used to fetch spatial names and alternative localized names, should be True at first run
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
getSpatialNamesFromOSM=False

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If set to True, details per PPN will be saved as separate JSON files in "jsonWebDir"
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
serializePPNLookup2JSON=False
jsonWebDir="./web/data/ppn/"

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If set to True, the raw features created by the Java application will be read from disk and converted for further usage, must be True at first run
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
reinterpretVisualWordRawFeatures=False

# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# If set to True, a HTTP webserver will be started after all cells have been processed to show the results
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
launchHTTPServer=False

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 40)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")

from bs4 import BeautifulSoup
from collections import OrderedDict # provides the ordered dictionary
import re # for regular expressions used below
import urllib # to read from URLs
import json
import networkx as nx # network analysis
from networkx.readwrite import json_graph
import itertools
import os.path
from datetime import datetime # for time measurement
import sys
import os
import pickle
import subprocess as subp
import gzip
import math
import codecs

from jellyfish import jaro_distance, jaro_winkler, hamming_distance, levenshtein_distance
import scipy.cluster.hierarchy as scipycluster
from sklearn.feature_extraction.text import TfidfVectorizer

from skimage import io, exposure
from scipy.spatial import distance
# import the k-means algorithm
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin,pairwise_distances_argmin_min, pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances

# OAI
from sickle import Sickle

# image handling
from PIL import Image

# geo stuff
import googlemaps
from geopy.distance import vincenty
import geojson as gj

def printLog(text):
    now=str(datetime.now())
    print("["+now+"]\t"+text)
    # forces to output the result of the print command immediately, see: http://stackoverflow.com/questions/230751/how-to-flush-output-of-python-print
    sys.stdout.flush()
    
def pickleCompress(fileName,pickledObject):
    printLog("Pickling to '%s'" %fileName)
    f = gzip.open(fileName,'wb')
    pickle.dump(pickledObject,f)
    f.close()
    printLog("Pickling done.")
    
def pickleDecompress(fileName):
    #restore the object
    printLog("Depickling from '%s'" %fileName)
    f = gzip.open(fileName,'rb')
    pickledObject = pickle.load(f)
    f.close()
    printLog("Depickling done.")
    return pickledObject

if not os.path.exists(downloadDir):
        os.makedirs(downloadDir)
if not os.path.exists("./picklez/"):
        os.makedirs("./picklez/")
if not os.path.exists("./figures/"):
        os.makedirs("./figures/")
if not os.path.exists("./graphs/"):
        os.makedirs("./graphs/")
if not os.path.exists("./graphs/generics/"):
        os.makedirs("./graphs/generics/")        
if not os.path.exists("./html/"):
        os.makedirs("./html/")
if not os.path.exists("./web/"):
        os.makedirs("./web/")
if not os.path.exists("./web/qa/"):
        os.makedirs("./web/qa/")
if not os.path.exists("./web/data/"):
        os.makedirs("./web/data/")
if not os.path.exists("./web/data/ppn/"):
        os.makedirs("./web/data/ppn/")
if not os.path.exists("./web/data/layers/"):
        os.makedirs("./web/data/layers/")

The only missing package is most likely Jellyfish, which provides support for string matching and offers such phonetic distance functions that we will need below. For further details, see the [Jellyfish homepage](https://pypi.python.org/pypi/jellyfish). The package can be installed by running the following command.

## Retrieving Data
### Connecting to the OAI-conform Repository and Downloading Metadata Records

The next cell connects to the Berlin State Library OAI-PMH server to download metadata records in the [Dublin Core format](http://dublincore.org/).
The records are then saved locally to work with them later. For the sake of simplicity, the saving relies on the pickling mechanism of Python - its built-in object serialization method. In a productive use case, you might reach the limits of the pickle package because of the resulting file size. A viable alternative is [HDF5](https://www.h5py.org/).

In [27]:
repository_address = 'http://sedici.unlp.edu.ar/oai/snrd'
#repository_address = 'http://repositorio.filo.uba.ar/oai/request'

In [28]:
savedRecords=[]
if allowDownloads:
    # connect to a metadata repository
    #sickle = Sickle('http://digital.staatsbibliothek-berlin.de/oai')
    sickle = Sickle(repository_address)
    # for debugging purposes you might want to uncomment the following commands
    # get the sets from the data provider connected to
    #sets = sickle.ListSets()
    #print the returned sets including their identifiers
    #print("Sets provided by data provider\n* * * * * * * * * * * * * * * * * * * * * ") # \n creates a new line
    #for s in sets:
        #print("'"+s.setName+"' accessible via: '"+s.setSpec+"'")

    # get the records from this repository's specific document set 'DC_krieg.1914.1918' (documents related to World War I) 
    # using Dublin Core format 
    #records = sickle.ListRecords(metadataPrefix='oai_dc', set='DC_allf')
    records = sickle.ListRecords(metadataPrefix='oai_dc')
    printLog("Starting OAI-PMH record download...")
    # initialize some variables for counting and saving the metadata records
    savedDocs=0
    # 2:15 h for 100k
    maxDocs=100 # 100 is just for testing, for more interesting results increase this value to 1000. ATTENTION! this will also take more time for reading data.

    # save the records locally as we don't want to have to rely on a connection to the OAI-PMH server all the time
    # iterate over all records until maxDocs is reached
    # ATTENTION! if you re-run this cell, the contents of the savedRecords array will be altered!
    for record in records:
        # get the PPN of the digitized work as the DC identifier will reference different (analog) manifestations
        tokens=record.header.identifier.split(":")
        tokens
        for t in tokens:
            if t.startswith("PPN"):
                # the array building simply fixes the issue that all DC field come as arrays
                record.metadata['PPN']=[t]
        
        # check if we reach the maximum document value
        if savedDocs<maxDocs:
            savedDocs=savedDocs+1
            # save the current record to the "savedRecords" array
            savedRecords.append(record.metadata)
            if savedDocs%1000==0:
                printLog("Downloaded %d of %d records."%(savedDocs,maxDocs))
        # if so, end the processing of the for-loop
        else:
            break # break ends the processing of the loop

    printLog("Finished OAI-PMH download of "+str(len(savedRecords))+" records.")
    pickle.dump( savedRecords, open( "save_120k_dc_all.pickle", "wb" ) )
else:
    printLog("Loading OAI-PMH records from disk...")
    savedRecords=pickle.load( open( "save_120k_dc_all.pickle", "rb" ) )
    printLog("Done.")

[2020-08-11 00:18:01.238000]	Starting OAI-PMH record download...
[2020-08-11 00:18:01.394000]	Finished OAI-PMH download of 100 records.


In [47]:
record = savedRecords[34]
record["identifier"]

['http://sedici.unlp.edu.ar/handle/10915/1393',
 'https://doi.org/10.35537/10915/1393',
 'http://digital.cic.gba.gob.ar/handle/11746/3445']

In [ ]:
http://sedici.unlp.edu.ar/bitstream/handle/10915/1377/Documento_completo.pdf?sequence=1&isAllowed=y

In [44]:
record

{'contributor': ['Christiansen, Carlos'],
 'creator': ['Cancelo, Gustavo Indalecio Eugenio'],
 'date': ['1996'],
 'description': [u'En esta tesis se realiza un estudio sobre las posibilidades de la electr\xf3nica para modelar unidades y sistemas neuronales. En particular, se proponen implementaciones de redes neuronales anal\xf3gicas. La primera a nivel microelectr\xf3nico, comprende el dise\xf1o de una neurona computacional con funci\xf3n de activaci\xf3n gaussiana. Primeramente, se demostrar\xe1 que las unidades computacionales con funci\xf3n de activaci\xf3n gaussiana forman una base para aproximadores universales de funciones continuas y discontinuas medibles. La segunda abarca la realizaci\xf3n de una neurocomputadora basada en un dispositivo neuronal anal\xf3gico. Este sistema aprovecha toda la potencia de c\xf3mputo de un procesador neuronal capaz de procesar 300 mil patrones anal\xf3gicos por segundo. El ambiente desarrollado permite que dicho procesador se convierta en el cora

In [ ]:
# uncomment this cell if you want to split the image download to two different computers
# in this case, one computer will use "even_dc_all.pickle" while the other will use "odd_dc_all.pickle" in the next cell

#availableKeys=dict()
#evenRecords=[]
#oddRecords=[]

#for i,r in enumerate(savedRecords):
#    for k in r.keys():
#        if not k in availableKeys:
#            availableKeys[k]=1
#        else:
#            availableKeys[k]=availableKeys[k]+1
#    if i%2==0:
#        evenRecords.append(r)
#    else:
#        oddRecords.append(r)

#pickle.dump( evenRecords, open( "even_dc_all.pickle", "wb" ) )
#pickle.dump( oddRecords, open( "odd_dc_all.pickle", "wb" ) )

In the next step, all images are downloaded, downscaled and converted to the JPEG format.

In [ ]:
# uncomment this line if you are continuing the execution of this notebook at a later point in time
#savedRecords=pickle.load( open( "save_120k_dc_all.pickle", "rb" ) )
tiffDownloadPath="https://ngcs.staatsbibliothek-berlin.de/?action=metsImage&format=jpg&metsFile=%PPN%&divID=PHYS_0001&original=true"
countSavedRecords=len(savedRecords)
printLog("Started image download and processing. This will take a while...")
#logFile = open("/Volumes/2TB_WD/sbb_images/downloadIssues.txt", "w")
logFile = open(downloadDir+"/downloadIssues.txt", "w")

# surpress PIL's DecompressionBombErrors and Warnings
Image.MAX_IMAGE_PIXELS = None

for i,record in enumerate(savedRecords):
    if i%1000==0:
        if allowDownloads:
            printLog("Downloaded %d of %d images."%(i,countSavedRecords))
    
    ppn=""
    for r in record["identifier"]:
        if r.startswith("PPN"):
            ppn=r
            break
    #if len(record["identifier"])>1:
    #    ppn=str(record["identifier"][1])
    #else:
    #    ppn=str(record["identifier"][0])
    ppnTIFF=ppn+".tif"
    ppnJPEGPAth=downloadDir+ppn+".jpg"
    #if "object" in record.keys() and allowDownloads:
    if allowDownloads:
        httpCode=200
        # prevent downloading of already present files
        if not os.path.isfile(ppnJPEGPAth) :
            tryDownload=True
            # check for the HTTP error code, maybe the file does not exist
            try:
                response=urllib.request.urlopen(tiffDownloadPath.replace("%PPN%",ppn))
            except urllib.error.HTTPError as ex:
                #print("Error with %s"%tiffDownloadPath.replace("%PPN%",ppn))
                httpCode=ex.code
                tryDownload=False
            if tryDownload:
                #if allowDownloads:
                urlinfo=urllib.request.urlretrieve(tiffDownloadPath.replace("%PPN%",ppn),downloadDir+ppnTIFF)
                img = Image.open(downloadDir+ppnTIFF)
                img.thumbnail((512,512))
                img.save(downloadDir+ppn+".jpg")
                if not keepTIFFs:
                    os.remove(downloadDir+ppnTIFF)
                #ret=subp.call(["mogrify", "-resize","512x512","-format", "jpg",downloadDir+ppnTIFF])
                #if ret!=0:
                #    print("Problem with mogrifying "+ppnTIFF)
                #    logFile.write("[MOGRIFY]: %s \n%s\n\n" % (str("Problem with mogrifying "+ppnTIFF),str("Downloaded from: "+record["object"][0])))
                #ret=subp.call(["rm",downloadDir+ppnTIFF])
                #if ret!=0:
                #    print("Problem with removing "+ppnTIFF)
                #    logFile.write("[REMOVAL]: %s\n\n" % "Problem with removing "+ppnTIFF)
            else:
                print("Problem with accessing "+ppnTIFF+ " @ "+tiffDownloadPath.replace("%PPN%",ppn)+" due to HTTP code: "+str(httpCode))
                logFile.write("[HTTP]: %s\n\n" % "Problem with accessing "+tiffDownloadPath.replace("%PPN%",ppn))
                logFile.write("\tHTTP Code: "+str(httpCode)+"\n")
                #logFile.write(str(urlinfo[1])+"\n\n")
    else:
        logFile.write("[OBJECT key missing]: %s\n\n" % str(record))
logFile.close()
print("\n")
if allowDownloads:
    printLog("Finished image download and processing.")
else:
    printLog("Skipped image download and processing.")

Please note that images will not be downloaded for some metadata records. This is no error because periodicals or multivolume work do not come with presentation images.

### Creating a Dataframe from the Metadata Records

In [ ]:
# load the records
printLog("Loading pickled records...")
# uncomment this line if you are continuing the execution of this notebook at a later point in time
savedRecords=pickle.load( open( "save_120k_dc_all.pickle", "rb" ) )
printLog("Finished loading pickled records.")

availableKeys=dict()

# check for all keys present in the previously downloaded dataset
for i,r in enumerate(savedRecords):
    for k in r.keys():
        if not k in availableKeys:
            availableKeys[k]=1
        else:
            availableKeys[k]=availableKeys[k]+1
    
print(availableKeys)

# create a dictionary for the records
values=dict()
# take the keys as they have found within the downloaded OAI records
keys=availableKeys.keys()
# for every metadata field, create an empty array as the content of the dictionary filed under the key 'k'
for k in keys:
    values[k]=[]
# in addition, store the PPN (the SBB's unique identifier for digitized content)    
#values["PPN"]=[]

# iterate over all saved records
for record in savedRecords:
    # we cannot iterate over the keys of record.metadata directly because not all records cotain the same fields,...
    for k in keys:
        # thus we check if the metadata field 'k' has been created above
        if k in values:
            # append the metadata fields to the dictionary created above
            # if the metadata field 'k' is not available input "None" instead
            #values[k].append(record.get(k,["None"])[0].encode('ISO-8859-1'))
            if k in record:
                value=record.get(k)[0]
                if value:
                    if value.isdigit():
                        value=int(value)
                    else:
                        #p27 value=value.encode('ISO-8859-1')
                            #value=value.encode('ISO-8859-1').decode("utf-8", "backslashreplace")
                        value=value
                    values[k].append(value)
                else:
                    values[k].append(np.nan)
            else:
                values[k].append(np.nan)
# create a data frame from the 
#p27 df=pd.DataFrame(pd.to_numeric(values,errors='coerce'))
df=pd.DataFrame(values)
df['date']=pd.to_numeric(df['date'],errors='ignore',downcast='integer')
#df=pd.DataFrame(values)
#df=df.convert_objects(convert_dates=False, convert_numeric=True, convert_timedeltas=False, copy=True)
df.shape

In [ ]:
# creating a PPN list in CSV format is handy if you want to use it with sbbget from StabiHacks
#df.PPN.to_csv("120k_ppn_list.csv",index=False)

# in case you want to use this data frame somewhere else, here are some sample serializations
#printLog("Serializing data frame...")
# Microsoft Excel
#df.to_excel("120k_sbb_sample.xlsx")
# HDF5 format
#df.to_hdf("120k_sbb_sample.hdf5",key="sbb")
#printLog("Done.")

In [ ]:
# if you want to continue from here, it might be a good idea to 
df=pd.read_excel("ppn_records_146000.xlsx")
df.shape

## Inspecting Data

In [ ]:
df.head()

In [ ]:
df[df.PPN.isnull()].count()

Usually, we know that some columns should only contain unique values, e.g., the PPN column. Hence, it is a good starting point for the analysis to validate this hypothesis with the help of the following function.

In [ ]:
# checks for unique values in each column
def uniqueValues(currentDataFrame):
    colNames=currentDataFrame.columns.values.tolist()
    for colName in colNames:
        print(colName+";\t\t unique values:\t"+str(len(currentDataFrame[colName].unique()))+ "\t total count: "+str(currentDataFrame[colName].count()))

uniqueValues(df)

Unfortunately, not all entries in the PPN column are unique. In other works, some works appear more than one time in the retrieved dataset. This is an observation we should definitely take care of later.

* https://www.maxmind.com/en/free-world-cities-database
* http://www.geonames.org/export/
* http://www.opengeocode.org/download.php#cities
* https://en.wikipedia.org/wiki/Lists_of_cities_by_country

### Classification of Data with the Help of Regular Expressions

As our last observation made us sceptical about the general data quality, we should analyse the data in more detail. Regular expressions are a handy means in order to analyse columns that we expect to follow a certain internal structure or pattern.

The next cell creates various regular expression to discover positive or negative numbers, different date ranges, emails, ISBN numbers or the like.


In [ ]:
# regular expressions taken from: http://stackoverflow.com/questions/1449817/what-are-some-of-the-most-useful-regular-expressions-for-programmers
# extended by David Zellhöfer

patterns=dict()
#^ beginning of string, $ end of string, \d digits +: once or more times
patterns["positiveInteger"]="^\d+$"
patterns["negativeInteger"]="^-\d+$"
patterns["generalInteger"]="^-?\d+$"
patterns["positiveFloat"]="^\d*\.\d+$"
patterns["negativeFloat"]="^-\d*\.\d+$"
patterns["generalFloat"]="^-?\d*\.\d+$"
patterns["positiveGermanFloat"]="^\d*,\d+$"
patterns["negativeGermanFloat"]="^-\d*,\d+$"
patterns["generalGermanFloat"]="^-?\d*,\d+$"
# Date (dd mm yyyy, d/m/yyyy, etc.), in range 1000-2099 without proper February handling
patterns["dateVariant"]="^([1-9]|0[1-9]|[12][0-9]|3[01])\D([1-9]|0[1-9]|1[012])\D(1[0-9][0-9][0-9]|20[0-9][0-9])$"
patterns["year"]="^(1[0-9][0-9][0-9]|20[0-9][0-9])$"
patterns["ancientYear"]="^([0-1]?[0-9][0-9][0-9]|20[0-9][0-9])$"
patterns["century"]="^(1[0-9][Xx][Xx]|20[Xx][Xx])$"
patterns["ancientCentury"]="^([0-1]?[0-9][Xx][Xx]|20[Xx][Xx])$"
patterns["decade"]="^(1[0-9][0-9][Xx]|20[0-9][Xx])$"
patterns["ancientDecade"]="^([0-1]?[0-9][0-9][Xx]|20[0-9][Xx])$"
# year range with splitter "- / :", the splitter can be surrounded by an arbitrary amount of whitespaces (\s)
patterns["rangeYear"]="^\s*(1[0-9][0-9][0-9]|20[0-9][0-9])\s*(\-|\/|:)\s*(1[0-9][0-9][0-9]|20[0-9][0-9])\s*$"
patterns["rangeCentury"]="^\s*(1[0-9][Xx][Xx]|20[Xx][Xx])\s*(\-|\/|:)\s*(1[0-9][Xx][Xx]|20[Xx][Xx])\s*$"
patterns["rangeAncientYear"]="^\s*([0-1]?[0-9][0-9][0-9]|20[0-9][0-9])\s*(\-|\/|:)\s*(1[0-9][0-9][0-9]|20[0-9][0-9])\s*$"
patterns["rangeAncientCentury"]="^\s*([0-1]?[0-9][Xx][Xx]|20[Xx][Xx])\s*(\-|\/|:)\s*(1[0-9][Xx][Xx]|20[Xx][Xx])\s*$"
patterns["rangeYear2Digit"]="^\s*(1[0-9][0-9][0-9]|20[0-9][0-9])\s*(\-|\/|:)\s*([0-9][0-9])\s*$"
patterns["rangeDateVariant"]="^\s*([1-9]|0[1-9]|[12][0-9]|3[01])\D([1-9]|0[1-9]|1[012])\D(1[0-9][0-9][0-9]|20[0-9][0-9])\s*(\-|\/|:)\s*([1-9]|0[1-9]|[12][0-9]|3[01])\D([1-9]|0[1-9]|1[012])\D(1[0-9][0-9][0-9]|20[0-9][0-9])\s*$"

patterns["email"]="^[_]*([a-z0-9]+(\.|_*)?)+@([a-z][a-z0-9-]+(\.|-*\.))+[a-z]{2,6}$"
patterns["domain"]="^([a-z][a-z0-9-]+(\.|-*\.))+[a-z]{2,6}$"
patterns["url"]="^https?\:\/\/[a-zA-Z0-9.-]+\.[a-zA-Z]{2,3}\/?$"
patterns["ipv4"]="^(?:\d{1,3}\.){3}\d{1,3}$"
patterns["rgbHex"]="^#([a-fA-F0-9]{6}|[a-fA-F0-9]{3})$"
patterns["generalHex"]="^#[a-fA-F0-9]*$"
 
patterns["isbnPrefix"]="^ISBN(-1(?:(0)|3))?:?\x20(\s)*[0-9]+[- ][0-9]+[- ][0-9]+[- ][0-9]*[- ]*[xX0-9]$"
patterns["isbn"]="^[0-9]+[- ][0-9]+[- ][0-9]+[- ][0-9]*[- ]*[xX0-9]$"
patterns["NaN"]="^[Nn][Aa][Nn]$"

Now, it is time to find out if all date columns match the patterns specified above.

In [ ]:
rowCount=0
histogram=dict()
for row in df.iterrows():
    rowCount=rowCount+1
    readDate=str(row[1]["date"])
    matchedOnce=False
    for key in patterns:
        p=re.compile(patterns[key])
        m = p.search(readDate)
        if m:
            if not key in histogram:
                histogram[key]=0
            histogram[key]=histogram[key]+1
            matchedOnce=True
        else:
            pass
    if not matchedOnce:
        print("No matches at all: "+row[1]["PPN"]+"\t for: "+str(readDate))
print("Row count: "+str(rowCount))
print(histogram)   

In [ ]:
plt.plot(*zip(*sorted(histogram.items())))


As expected, the matching yields a "sediment" of extreme cases that need special treatment, e.g., "1756 $ [ca. 1756]". By iterating this process, we could easily address all patterns present in the dataset.

### Inspecting Spatial Locations

Because the dataset contains a lot of historical works, the _spatial_ columns deserves further attention. To get a feeling for the data, we will inspect its contents.

In [ ]:
df.coverage.head(30)

Unfortunately, the spatial column does not only contain cities.  Sometimes, more than one city or a even a country is contained. Another variant uses square brackets. Additionally, specific bibliographic terminology indicate that no place of publication ([s.l.]=sine loco=without place or o.O. as its German equivalent) could be determined.

### Combining Data Cleansing Utilities

In [ ]:
class DataCleaner:
    # matches alphanumeric character and the underscore at the beginning of the string
    #Unicode flag is needed because of Asian character sets otherwise such signs would be considered as non-alphanumeric
    regEx_AlphaNum=re.compile("^\w",re.UNICODE)
    # checks for surrounding []; will match almost everything but Asian characters
    regEx_BracketText=re.compile("^\[[\w\?\.,\sßÄäÖöÜü]*\]",re.UNICODE)
    # checks for typical spellings of the "sine loco" abbreviation "s. l."
    regEx_SineLoco=re.compile("[sSoO]\s?\.\s?[lLoO]\s?\.?\s?",re.UNICODE)
    
    def __init__(self):
        pass
    
    def cleanAncientYearStrict(self,readData):
        if type(readData)==float:
            if not readData:
                return 0
            else:
                return readData
        else:
            p=re.compile(patterns["ancientYear"])
            m = p.search(str(readData))
            if m:
                firstAppearance=m.group()
                return firstAppearance
            else:
                return np.nan
            
    def cleanSpatialText(self,readData):
        returnedString=""
        # just in case we did not get a string, we use brute force and return NaN
        if type(readData)==float:
            return ""
        else:
            #readData=str(readData)
            m = self.regEx_AlphaNum.search(readData)
            # if the string does start with a bracket...
            if not m:
                #print "No matches at all: "+row[1]["PPN"]+"\t for: "+str(readData)
                m2 = self.regEx_BracketText.search(readData)
                if m2:
                    matchedGroup=m2.group()
                    #print "\tMatch: "+matchedGroup
                    m3=self.regEx_SineLoco.search(matchedGroup)
                    if m3:
                        #print "\tMatched Sine Loco: "+str(m3.group())
                        return ""
                    else:
                        matchedGroup=matchedGroup.replace("[","").replace("]","")
                        #print "\tFinal string: "+matchedGroup
                        returnedString=matchedGroup
            # otherwise, it may still be a "sine loco"
            else:
                m3=self.regEx_SineLoco.search(readData)
                if m3:
                    #print "\tMatched Sine Loco: "+str(m3.group())
                    return ""
                else:
                    # in any case, there might be brackets left
                    returnedString=readData.replace("[","").replace("]","")
        
        # remove variants of "u.a."            
        regex = re.compile("[uU]\.\s?[aA]\.\s?",re.UNICODE)
        returnedString=regex.sub("",returnedString)
        return returnedString

### Cleaning Data

In [ ]:
dc=DataCleaner()

#for row in df.iterrows():
#    print dc.cleanSpatialText(str(row[1]["spatial"]))
    
df['spatialClean'] = df.coverage.apply(dc.cleanSpatialText)
df['dateClean'] = df.date.apply(dc.cleanAncientYearStrict)

gate 2

In [ ]:
df.head(30)

In [ ]:
uniqueValues(df)

In [ ]:
#py27 df.sort_values(by="date")

# http://stackoverflow.com/questions/40032341/pandas-sort-dataframe-by-column-with-strings-and-integers
df.groupby(df.date.apply(type) != str).apply(lambda g: g.sort('date')).reset_index(drop = True)

## Using Clustering for Further Data Cleansing
### Example of The Things We Are Up To

In [ ]:
words = u'Berlin Balin Cölln Köln'.split()
print(words)

print("Number of words: %i" % len(words))
for i,val in enumerate(words):
    print(str(i)+":\t "+str(val.encode('utf-8')))
    

First, we are interested in finding out whether the Jaro distance is really a distance. As you might remember from your studies, a (normalized) distance yields 1.0 a dissimilar object. Hence, the distance between "Berlin" and "Berlin" should be 0.

In [ ]:
jaro_distance(u'Berlin',u'Berlin')

Obviously, it is not a distance. Therefore, we have to subtract the future results of __jaro_distance()__ from 1.0 to get a real distance score.

In [ ]:
# http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.triu_indices.html
# 2nd parameter:
# Diagonal above which to zero elements. k = 0 (the default) is the main diagonal, k < 0 is below it and k > 0 is above."""
# r= Return the indices for the upper-triangle of an (n, m) array. da m nicht angegeben ist, wird n=m angenommen
# m is not passed, hence m=n

# sagen, dass die matrix square ist!
r=np.triu_indices(n=len(words), k=1)
r

what does this mean?
$$
A=
\begin{pmatrix}
a_{0,0} & \underline{a_{0,1}} & \underline{a_{0,2}} & \underline{a_{0,3}} \\
\cdot & a_{1,1} & \underline{a_{1,2}} & \underline{a_{1,3}} \\
\cdot & \cdot & a_{2,2} & \underline{a_{2,3}} \\
\cdot & \cdot & \cdot & a_{3,3}
\end{pmatrix}
$$

how to interpret: $a_{0,1}$, i.e., the difference between "Berlin" and "Balin"

In [ ]:
def d_demo(coord):
    print(coord)
    i, j = coord
    # 1- wg. Distanz (see above)
    return 1-jaro_distance(words[i], words[j])

In [ ]:
# http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.set_printoptions.html
np.set_printoptions(precision=4)

# axis (3rd parameter): 0= along y axis, 1= along x axis
r2=np.apply_along_axis(d_demo, 0, r)
r2

what does this mean for our matrix?
$$
A=
\begin{pmatrix}
a_{0,0} & \underline{0.1778} & \underline{0.4222} & \underline{0.3889} \\
\cdot & a_{1,1} & \underline{0.4} & \underline{0.3667} \\
\cdot & \cdot & a_{2,2} & \underline{0.2167} \\
\cdot & \cdot & \cdot & a_{3,3}
\end{pmatrix}
$$

how to interpret: $a_{0,1}$, i.e., the difference between "Berlin" and "Balin" is 0.17777778.

why not the elements on the diagonal? because...



### Working with Real Data

first, we have to define the distance function

In [ ]:
def d(coord):
    #print(coord)
    i, j = coord
    #py27 return 1-jaro_distance(unicode(str(words[i]), 'utf-8'), unicode(str(words[j]), 'utf-8'))
    return 1-jaro_distance(words[i],words[j])

In [ ]:
df3=df#pd27 .sort_values(by="date")#.head(100)

uniqueSpatials=df3["spatialClean"].unique()
words=None
if demoClustering:
    words=uniqueSpatials[:100] # only consider the first 100 elements for performance reasons
else: # during normal operation, we want to consider all unique spatial names...
    words=uniqueSpatials
r=np.triu_indices(len(words), 1)

the next step will take some time, hence we limited the number of spatial labels before (3-4 min.)

In [ ]:
printLog("Started calculation of distance matrix for %i words..."%len(words))
# _ is the last evaluated value in an interactive shell
# axis (3rd parameter): 0= along y axis, 1= along x axis
r2=np.apply_along_axis(d, 0, r)
printLog("Finished calculations.")

agglomeratives Clustering (hier nearest points, bottum-up) im gegensatz zu divisiven (top-down), das Beispiel hier nutzt den nearest point algorithm (Formel siehe https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.linkage.html)

In [ ]:
Z=scipycluster.linkage(r2)

if not demoClustering:
    pickle.dump( Z, open( "cluster_hierarchy_linkage_result_without_name_clustering.pickle", "wb" ) )

#Z
# scientific notation erklären

i müsste Zeile sein

A 4 by (n-1) matrix Z is returned. At the i-th iteration, clusters with indices Z[i, 0] and Z[i, 1] are combined to form cluster n + i. A cluster with an index less than n corresponds to one of the n original observations. The distance between clusters Z[i, 0] and Z[i, 1] is given by Z[i, 2]. The fourth value Z[i, 3] represents the number of original observations in the newly formed cluster.

mehr infos: https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/


In [ ]:
#Z=pickle.load( open( "cluster_hierarchy_linkage_result.pickle", "rb" ) )
if demoClustering:
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('sample index')
    plt.ylabel('distance')
    scipycluster.dendrogram(
        Z,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=16.,  # font size for the x axis labels
    )
    plt.show()


* fcluster macht aus der clusterhierarchie wiederum "flat clusters"

In [ ]:
clusters=scipycluster.fcluster(Z, t=0.1,criterion="distance")
# 2. parameter ist abhängig von der clustering strategie, -> cophenetic distance
# see: http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html
# An array of length n. T[i] is the flat cluster number to which original observation i belongs.


# https://stat.ethz.ch/R-manual/R-devel/library/stats/html/cophenetic.html
# https://en.wikipedia.org/wiki/Cophenetic
# wir erhalten für jedes unser elemente eine cluster ID
clusters

service functions that help to establish a linkage between the cluster IDs and human readable names

In [ ]:
def getWordIndex(word):
    return np.where(words==word)[0]

def getClusterID(data):
#for row in df3.iterrows():
    #data=row[1]["spatialClean"]
    #wordIndex=np.where(words==data)[0]
    #if data == u"奈良".encode('utf-8'):
    #    print "China!"
    #    wordIndex=getWordIndex(data)
    #    print wordIndex
    #    print clusters[wordIndex][0]
    wordIndex=getWordIndex(data)
    if wordIndex:
        return clusters[wordIndex][0]
    else:
        return ""

### Inspecting the Clustered Results Deeper

In [ ]:
#
Z_huge=pickle.load( open( "cluster_hierarchy_linkage_result_without_name_clustering.pickle", "rb" ) )
uniqueSpatials=df3["spatialClean"].unique()
words=uniqueSpatials
len(words)

In [ ]:
clusters=scipycluster.fcluster(Z_huge, t=0.07,criterion="distance")
clusters

gate 3

In [ ]:
df3['spatialCluster'] = df3["spatialClean"].apply(getClusterID)
grp=df3.groupby("spatialCluster")
#print grp.groups.keys()
print("Number of clusters: %i" % len(grp.groups.keys()))

stichproben ...

In [ ]:
grp.get_group(clusters[getWordIndex("Berlin")][0])

shortcut, because we are only interested in the unique names within a cluster...

In [ ]:
grp.get_group(clusters[getWordIndex("Frankfurt/Oder")][0])["spatialClean"].unique()

semantisch sind die cluster natuerlich nicht korrekt...

a good time for inspecting all of our clusters' contents

In [ ]:
for key in grp.groups.keys():
    if key:
        print(key)
        print(grp.get_group(key)["spatialClean"].unique())

* synonym: ['Francofurti Ad Viadrum'] as a Latin translation of Frankfurt/Oder would be long to the cluster with ['Frankfurt/Main' 'Frankfurt/Oder' 'Frankfurt, Main' 'Frankfurt, O']
* however, Frankfurt/Main and Frankfurt/Oder are to differt cities
* ['Francofurti'] is in a 1-element cluster
* ['C\xc3\xb6lln an der Spree'] is a synonym for Berlin
* duplicate entries: ['Hallae Magdeburgicae  Hallae Magdeburgicae'] 1-gram und 2-gram vergleichen!
* auf Enthaltensein von Berlin prüfen
* St. Sankt Saint Bad als Präfix behandeln

In [ ]:
# backup of the dataframe as we are going to remove data from the spatialClean column
pickleCompress('./picklez/df3_unclean_spatials.picklez',df3)
# if you want to continue from here, uncomment the following line
#df3=pickleDecompress('./picklez/df3_unclean_spatials.picklez')

### Further Cleaning

on some occasions, we have to deal with spatialClean entries that contain multiple cities

In [ ]:
# PPN836134109 contains multiple cities
df3[df3.PPN=="PPN836134109"]

handle compound city names (only today's German cities to give an example)

* to avoid superfluous comparisons we split the city names by their leading character

In [ ]:
cityNameFile_DE = open('citynames.txt', 'r')
cityNGramsDE=dict()
lastLetter=""
for line in cityNameFile_DE:
    if not line.startswith("//"):
        r=line.replace("\n","").strip()
        if " " in r:
            #print r
            if not lastLetter==r[0].lower():
                lastLetter=r[0].lower()
                cityNGramsDE[lastLetter]=[]
            cityNGramsDE[lastLetter].append(r)
            #py27 cityNGramsDE[lastLetter].append(r.decode("utf-8"))

in case of multiple cities, we will only consider the first one

In [ ]:
# Various test cases of city names
#s1="Frankfurt, O"
#s2='Hallae Magdeburgicae  Hallae Magdeburgicae'
#s3="Leipzig  Paris  Petersburg  London"
#s4='Franckfurt  N\xc3\xbcrnberg  Leipzig'
#s5='Freiburg i.Br. ' # not matched correctly but okay
#s6='Frankfurt/Main' 
#s7='Frankfurt, Main'

#s8='Bad Nauheim'
#s9='Rottach-Egern am Tegernsee'
#s10='Egern a. Tegernsee'
#s11="Plancy-L'Abbaye"
#s12='Bad Nauheim Sankt'
#s13="Saint Tropez"
#s14="Sankt Augustin"
#s15="Sankt-Augustin"
#s16="St.-Whatever"
#s17="St. Whatever"

def pickFirstCity(testString):
# checks if the testString contains multiple cities separated by whitespaces and returns the first city respecting city name prefixes such as Saint, St. etc.
    #py27 testString=unicode(testString,"utf-8")
    #print type(testString)
    #testString=testString.decode('unicode-escape')
    
    # matches for whitespaces that are NOT preceded by the following signs: ", ; : \ / " denoted in the regex by (?<!...)
    regex = re.compile("(?<![,;:\\\/])\s*",re.UNICODE)

    # matches various city prefix such as Saint etc.
    spatialPrefixRegExes=[]
    spatialPrefixRegExes.append(re.compile("^[Bb][Aa][Dd]\s*",re.UNICODE))
    spatialPrefixRegExes.append(re.compile("^[Ss][Aa][Nn][Kk][Tt][\s-]*",re.UNICODE))
    spatialPrefixRegExes.append(re.compile("^[Ss][Aa][Ii][Nn][Tt][\s-]*",re.UNICODE))
    spatialPrefixRegExes.append(re.compile("^[S][t]\.[\s-]*",re.UNICODE))
    spatialPrefixRegExes.append(re.compile("^[Dd][Ee][Nn]\s*",re.UNICODE))
    spatialPrefixRegExes.append(re.compile("^[Ne][Ee][Ww]\s*",re.UNICODE))

    #print "Tested string: >%s<" % testString
    
    if allowExactStringMatching:
        # checks whether the spatial is known to consist of more than one word
        # for the sake of efficiency we will only check for spatial names with the same leading character (see above)
        lowTestString=testString.lower()
        if len(lowTestString)>=1:
            currentLetter=lowTestString[0]
            if currentLetter in cityNGramsDE:
                for cityName in cityNGramsDE[currentLetter]:
                    foundIndex=lowTestString.find(cityName.lower())
                    #regexCity = re.compile("^\b"+cityName.lower()+"\b",re.UNICODE)
                    r=re.match("\\b"+cityName.lower()+"\\b",lowTestString)
                    #if foundIndex==0:
                    if r is not None:
                        #print "Compound city: "+cityName+" @ "+str(foundIndex)
                        return cityName
    
    # general purpose splitting
    foundSpatialPrefix=False
    for i,r in enumerate(spatialPrefixRegExes):
        m = r.search(testString)
        if m:
            #print "Prefix %i" %i
            foundSpatialPrefix=True
    #print type(testString)
    m = regex.split(testString)
    
    if foundSpatialPrefix:
        if len(m)>1:
            return m[0]+" "+m[1]
    else:
        return m[0]

print(pickFirstCity('Bad Königshofen im Grabfeld'))   

bevor man die erste stadt auswählt muss man, sich die einträge mit mehr als einer stadt speichern.

In [ ]:
counter=0
maxItems=df3.shape[0]

multipleCitiesPPN=dict()

printLog("Processing multiple spatial names entries...")
for row in df3.iterrows():
    ppn=str(row[1]["PPN"])  
    spatialC=str(row[1]["spatialClean"])
    multipleCitiesPPN[ppn]=[]
    
    counter=counter+1
    if counter%10000==0:
        printLog("\tProcessed %i items of %i"%(counter,maxItems))
                
    if spatialC:
        origSpatialClean="<"+spatialC+"> ("+ppn+")"
        f=pickFirstCity(spatialC)
        loopCount=1
        while f:
            #print f
            multipleCitiesPPN[ppn].append(f)
            #py27 spatialC=spatialC.decode("utf-8")
            spatialC=re.sub('\s+',' ',spatialC.replace(f,u"")).strip()
            #py27 spatialC=spatialC.encode("utf-8")
            if spatialC:
                #print "<"+f.encode("utf-8")+"> | <"+spatialC+">"
                #f=spatialC
                f=pickFirstCity(spatialC)
            else:
                #print origSpatialClean
                break
            loopCount=loopCount+1
            if loopCount>15:
                printLog("Max. loop count reached @ "+origSpatialClean+ " with last f of: <"+f+">")
                break
        #if f:
            #spatialC=spatialC.decode("utf-8")
            #spatialC=spatialC.replace(f,u"").strip()
            #spatialC=spatialC.encode("utf-8")
            #if spatialC:
                #print origSpatialClean
                #print "<"+f.encode("utf-8")+"> | <"+spatialC+">"
                #counter=counter+1
                #if counter==10:
                    #break
    #print ppn
    #break
printLog("Done.")
pickleCompress("./picklez/multipleSpatialNamesPerPPN.picklez",multipleCitiesPPN)

display the PPNs with multiple spatial names

In [ ]:
for ppn in multipleCitiesPPN:
    array=multipleCitiesPPN[ppn]
    if len(array)>1:
        #py27 print(ppn+":\t"+"; ".join(array).encode("utf-8"))
        print(ppn+":\t"+"; ".join(array))
        #for e in array:
            #print e.encode("utf-8")

* interessante PPN: PPN780104447; PPN771083963; PPN792355296; PPN735012342
* Umgang mit ? klären
* Einträge des Arrays übeprüfen: Propositionen (auch Mehrwerte-Komposita wie "an der"), Nicht-Ortsnamen (z.B. Poststempel)

* finally, we can extract the first cities

* next cell might take ca. 3 min. with list-based exact matching

In [ ]:
printLog("Detecting and picking first cities...")
uniqueSpatials=df3["spatialClean"].unique()
beforeClusterClean=len(uniqueSpatials)
df3["spatialClean"]=df3["spatialClean"].apply(pickFirstCity)
printLog("Done.")

if allowExactStringMatching:
    pickleCompress("./picklez/df3_exactmatch_cities.picklez",df3)
else:
    df3=pickleDecompress("./picklez/df3_exactmatch_cities.picklez")
printLog("Done.")

In [ ]:
df3[df3.coverage.notnull()].head(20)

gate 4

check our results

In [ ]:
uniqueSpatials=df3["spatialClean"].unique()
afterClusterClean=len(uniqueSpatials)
words=uniqueSpatials
#words
print("Before cluster cleaning: %i" % beforeClusterClean)
print("After cluster cleaning: %i" % afterClusterClean)

In [ ]:
def d2(coord):
    #print "Altered d()"
    #print coord
    i, j = coord
    #print(str(type(words[i]))+" : "+str(type(words[j])))
    #py27 if not type(words[i])==unicode:
        #print "bumm "+ str(words[i])+" :"+str(i)
    if not words[i]:
        return 1.0
    #py27 if not type(words[j])==unicode:
        #print "bamm " + str(words[j])+" :"+str(j)
    if not words[j]:
        return 1.0
    dist=1 - jaro_distance(words[i],words[j]) # because jaro_distance is actually returning a similarity
    #print "%s vs. %s -> %f" %(words[i],words[j],dist)
    #return 1 - jaro_distance(unicode(str(words[i]), 'utf-8'), unicode(str(words[j]), 'utf-8'))
    return dist

def d3(coord):
    #print "Altered d()"
    #print coord
    i, j = coord
    #print str(type(words[i]))+" : "+str(type(words[j]))
    #py27 if not type(words[i])==unicode:
        #print "bumm "+ str(words[i])+" :"+str(i)
    if not words[i]:
        return 1.0
    #py27 if not type(words[j])==unicode:
        #print "bamm " + str(words[j])+" :"+str(j)
    if not words[j]:
        return 1.0
    dist=1 - jaro_winkler(words[i],words[j]) # because jaro_distance is actually returning a similarity
    #print "%s vs. %s -> %f" %(words[i],words[j],dist)
    return dist

vielleicht nötig, das ganze außerhalb des jupyter-notebooks auszuführen, wenn es time out probleme (siehe console) gibt

In [ ]:
if not demoClustering:
    printLog("Calculating Jaro distances...")
    r=np.triu_indices(afterClusterClean, 1)
    r2=np.apply_along_axis(d2, 0, r)
    Z=scipycluster.linkage(r2)
    pickle.dump( Z, open( "cluster_hierarchy_linkage_result.pickle", "wb" ) )
    
    # Jaro-Winkler
    printLog("Calculating Jaro-Winkler distances...")
    r=np.triu_indices(afterClusterClean, 1)
    r3=np.apply_along_axis(d3, 0, r)
    Z3=scipycluster.linkage(r3)
    pickle.dump( Z3, open( "cluster_hierarchy_linkage_jw_result.pickle", "wb" ) )
    #pickle.dump( r2, open( "r2.pickle", "wb" ) )
    printLog("Distance matrices created. Done.")
    # (END_HERE_FOR_LINKAGE)

### Working with the "Final" Data

In [ ]:
## load stuff
printLog("Loading final data...")
# Jaro
#Z_huge=pickle.load( open( "cluster_hierarchy_linkage_result.pickle_bak", "rb" ) )
# Jaro-Winkler
Z_huge=pickle.load( open( "cluster_hierarchy_linkage_jw_result.pickle", "rb" ) )

#r2=pickle.load( open( "r2.pickle", "rb" ) )
printLog("Loading final data completed.")

group records by their cluster and display results

In [ ]:
uniqueSpatials=df3["spatialClean"].unique()
words=uniqueSpatials
print(len(words))
# Jaro
#clusters=scipycluster.fcluster(Z_huge, t=0.115,criterion="distance") # je höher d, desto mehr kommt in 1 cluster
# Jaro-Winkler
clusters=scipycluster.fcluster(Z_huge, t=0.095,criterion="distance") # je höher d, desto mehr kommt in 1 cluster

print(clusters)
df3['spatialCluster'] = df3["spatialClean"].apply(getClusterID)
grp=df3.groupby("spatialCluster")

print("Number of clusters: %i" % len(grp.groups.keys()))

# save the clusters' names (for simplicity, we just take the name of the cluster's first element)
clusterNames=dict()
maxKey=-1
for key in grp.groups.keys():
    if key:
        if key>maxKey:
            maxKey=key
        if not key in clusterNames:
            clusterNames[key]=grp.get_group(key)["spatialClean"].unique()[0]
            
        print(str(key)+" <"+clusterNames[key]+"> contains:\n\t"+str(grp.get_group(key)["spatialClean"].unique()))
print("Max key value: %i"%maxKey)

In [ ]:
grp.get_group(clusters[getWordIndex("Moskau")][0])["spatialClean"].unique()

In [ ]:
grp.get_group(clusters[getWordIndex("Bamberg")][0])["spatialClean"].unique()

In [ ]:
grp.get_group(clusters[getWordIndex("Berlin")][0])["spatialClean"].unique()

In [ ]:
grp.get_group(clusters[getWordIndex("Frankfurt")][0])["spatialClean"].unique()

In [ ]:
grp.get_group(clusters[getWordIndex("Hamburg")][0])["spatialClean"].unique()

In [ ]:
grp.get_group(clusters[getWordIndex("Uri")][0])["spatialClean"].unique()

writing out the spatial cluster names...

In [ ]:
def getClusterName(clusterID):
    if clusterID:
        if clusterID in clusterNames:
            r=clusterNames[clusterID]
            if r:
                return r.title()
        else:
            return np.nan
    else:
        return np.nan

df3['spatialClusterName'] = df3["spatialCluster"].apply(getClusterName)

manual corrections of the spatial cluster names (über Karte gesehen)
* Carlsruhe
* Crefeld
* Weymar
* Kjøbenhavn
* Leipzig, Dresden
* St. Peterburg
* Coblentz
* Halæ

wäre besser, das vor dem Clustern zu tun

it would be cumbersome to correct everything manually without further assistance as shown below

In [ ]:
df3.ix[df3["spatialClusterName"]=="Carlsruhe", "spatialClusterName"] = "Karlsruhe"
df3.ix[df3["spatialClusterName"]=="Crefeld", "spatialClusterName"] = "Krefeld"
df3.ix[df3["spatialClusterName"]=="Weymar", "spatialClusterName"] = "Weimar"
df3.ix[df3["spatialClusterName"]==u"Kjøbenhavn", "spatialClusterName"] = u"København"
df3.ix[df3["spatialClusterName"]=="Leipzig, Dresden", "spatialClusterName"] = "Leipzig"
df3.ix[df3["spatialClusterName"]=="St. Peterburg", "spatialClusterName"] = "Sankt Petersburg"
df3.ix[df3["spatialClusterName"]=="Coblentz", "spatialClusterName"] = "Koblenz"
df3.ix[df3["spatialClusterName"]==u"Halæ", "spatialClusterName"] = u"Halle"

gate 5

In [ ]:
# if there was no cluster name, just take the cleaned spatial name
df3.spatialClusterName.fillna(df3.spatialClean, inplace=True)
uniqueValues(df3)

* in the next step, we will extract all unique spatial cluster names for manual correction
* an Excel sheet is created which also contains statistical data about the name in order to reveal outliers

In [ ]:
dfValues={'SpatialClusterName':[],'Length':[],'NoOfElements':[],'Entries':[]}

sortedUniqueDF3=df3.sort_values(by="spatialClusterName")["spatialClusterName"].unique()

for uniqueName in sortedUniqueDF3:
    if uniqueName:
        length=str(len(uniqueName))
        entries=df3[df3["spatialClusterName"]==uniqueName]["spatialClean"].unique()
        noOfElements=str(len(entries))
        entriesString=",".join(entries)
        dfValues['SpatialClusterName'].append(uniqueName)
        dfValues['Length'].append(length)
        dfValues['NoOfElements'].append(noOfElements)
        dfValues['Entries'].append(entriesString)

spDF=pd.DataFrame(dfValues)
spDF = spDF[['SpatialClusterName', 'Length','NoOfElements','Entries']]
spDF['NoOfElements'] = pd.to_numeric(spDF['NoOfElements'])
spDF['Length'] = pd.to_numeric(spDF['Length'])
spDF['NewName']=""

stdLength=spDF.std()["Length"]
avgLength=spDF.mean()["Length"]
lowerLength=avgLength-stdLength
upperLength=avgLength+stdLength

stdNoOfElements=spDF.std()['NoOfElements']
avgNoOfElements=spDF.mean()['NoOfElements']
lowerNoOfElements=avgNoOfElements-stdNoOfElements
upperNoOfElements=avgNoOfElements+stdNoOfElements

def atypicalLength(x):
    if x < lowerLength or x > upperLength:
        return True
    else:
        return False

def atypicalNoOfElements(x):
    if x < lowerNoOfElements or x > upperNoOfElements:
        return True
    else:
        return False
    

spDF['AtypicalLength']=spDF['Length'].apply(atypicalLength)
spDF['AtypicalNoOfElements']=spDF['NoOfElements'].apply(atypicalNoOfElements)

spDF.to_excel("./spatialnames.xlsx",index=False)
spDF.shape

visualization of the results

In [ ]:
grpNoOfElements=spDF.groupby("NoOfElements")
grpNoOfElements.count().plot()
if saveFiguresAsPDF:
    plt.savefig('./figures/noelements_cluster.pdf')

In [ ]:
grpLength=spDF.groupby("Length")
grpLength.count().plot()
if saveFiguresAsPDF:
    plt.legend("")
    plt.ylabel("Amount")
    plt.xlabel("Length of Cluster Name")
    plt.savefig('./figures/length_spatialnames.pdf')

* "Poststempel", "Erscheinungsort", nicht entzifferbare Abkürzungen lassen sich so auffinden oder: St.Moritz Hotel, bzw. Kombinationen aus Verlagen und Orten; Signaturen/Referenzen: "Br117"; "VIII, 35"; (Budae,Budapest,Budapesten) wird als ein Cluster erkannt

* Excel weißt statistische Besonderheiten aus und ist alphabetisch anhand der Cluster sortiert, so kommt man in gut XX Minuten mit der Korrektur durch

* nicht alle Korrekturen sind manuell notwendig, da Ortsnamen auch Mehrsprachig in OSM oder GND(???) liegen
* Character Encoding -Probleme sind dort auch gut zu erkennen (Piešťany)

* consonant shift or sound changes

* durch die Aufbereitung in ca. 45 Minuten leistbar (inkl. Recherchen in Historischen Ortsnamen, OSM und Google)

Manual correction should be done in _spatialnamesCorrections.xlsx_. For illustration purposes, some correction have been made. Dots indicate rows for which the spatial name cluster will be removed.

In [ ]:
spDF_manuallyCleaned=pd.read_excel("spatialnamesCorrections.xlsx")
spDF_manuallyCleaned.head(20)

In [ ]:
df3[df3.spatialClusterName=="Aigen/Salzburg"].head() # returns PPN771101503 and/or PPN771101589

In [ ]:
df3[df3.spatialClusterName=="1"].head() # returns PPN662043146 and PPN688520944

## korrekturen einarbeiten; sagen, welche spalten wie ausgefüllt werden müssen; vorteil excel: leute kennen das tool und können sich alles anpassen, wie sie s brauchen

In [ ]:
uniqueValues(df3)

In [ ]:
# df3.ix[df3["spatialClusterName"]=="Carlsruhe", "spatialClusterName"] = "Karlsruhe"
printLog("Fixing spatial cluster names on the basis of manual corrections...")
counterRemoved=0
counterAltered=0
ignoredSpatialNamesLowerCase=[]

for row in spDF_manuallyCleaned.iterrows():
    #py27 newName=readDate=unicode(row[1]["NewName"])
    #py27 oldName=unicode(row[1]["SpatialClusterName"])
    newName=str(row[1]["NewName"])
    oldName=row[1]["SpatialClusterName"]
    
    if newName==".":
        #print("Removed: "+row[1]["SpatialClusterName"])
        df3.ix[df3["spatialClusterName"]==oldName, "spatialClusterName"] = ""
        ignoredSpatialNamesLowerCase.append(oldName.lower())
        counterRemoved=counterRemoved+1
    elif newName=="nan":
        pass
    else:
        #print("Changed: "+row[1]["SpatialClusterName"])
        df3.ix[df3["spatialClusterName"]==oldName, "spatialClusterName"] = newName
        counterAltered=counterAltered+1
        
printLog("Done.")
printLog("Removed clusters: %i"%counterRemoved)
printLog("Altered clusters: %i"%counterAltered)

gate 6

In [ ]:
uniqueValues(df3)

# spatialCluster-ID muss auch noch angepasst werden und der spatialClusterName-Anzahl entsprechen

Some debugging...

In [ ]:
df3[df3.PPN=="PPN771101503"].head()

In [ ]:
df3[df3.PPN=="PPN662043146"].head()

In [ ]:
pickleCompress('./picklez/clean_dataframe.picklez',df3)
pickleCompress('./picklez/ignoredSpatialNames.picklez',ignoredSpatialNamesLowerCase)

# Geodaten erzeugen

In [ ]:
# uncomment only needed if you resume the notebook
df4=pickleDecompress('./picklez/clean_dataframe.picklez')
#ignoredSpatialNamesLowerCase=pickleDecompress('./picklez/ignoredSpatialNames.picklez')

# otherwise just run this line:
#df4=df3

In [ ]:
# not tested with Python 3
if useGoogleMapsAPI:
    # idee ist, die Algorithmen von Google zu nutzen, um mit mitunter fehlerhaften, mehrsprachigen Ortsbezeichner umzugehen
    gmaps = googlemaps.Client(key=privateGoogleKey)
    # if you re-run this cell you might want to uncomment the following line
    latLng=dict()
    printLog("Fetching geolocations from Google Maps...")
    for row in df4.iterrows():
        if not row[1]["spatialClusterName"]:
            # in some cases, i.e., when the cluster was corrected manually above, 
            # we will ignore the spatialClean replacement
            loc=row[1]["spatialClean"]
            if loc:
                if not loc.lower() in ignoredSpatialNamesLowerCase:
                    printLog("Taking %s instead."%loc)
                else:
                    loc=None
        else:
            loc=row[1]["spatialClusterName"]
        if loc:
            if loc not in latLng:
                #printLog(loc)
                try:
                    retLL=gmaps.geocode(loc)
                    if len(retLL)>0:
                        latLng[loc]=retLL[0][u'geometry'][u'location']
                    else:
                        latLng[loc]=None
                except googlemaps.exceptions.ApiError:
                    printLog("Problem fetching: "+loc)
                    latLng[loc]=None
    printLog("Done.")

    triedAgain=[]
    printLog("Fetching geolocations from Google Maps a second time (aka the brute force failover solution)...")
    for row in df4.iterrows():
        if not row[1]["spatialClusterName"]:
            loc=row[1]["spatialClean"]
        else:
            loc=row[1]["spatialClusterName"]
        if loc:
            if latLng[loc]==None:
                if loc not in triedAgain:
                    #printLog(loc)
                    triedAgain.append(loc)
                    try:
                        retLL=gmaps.geocode(loc)
                        if len(retLL)>0:
                            latLng[loc]=retLL[0][u'geometry'][u'location']
                        else:
                            latLng[loc]=None
                    except googlemaps.exceptions.ApiError:
                        printLog("Problem fetching: "+loc)
                        latLng[loc]=None
    printLog("Done.")
    pickleCompress('./picklez/geo_coordinates.picklez',latLng)
else:
    printLog("Skipping Google Map API calls.")

alternative with OpenStreetMap http://wiki.openstreetmap.org/wiki/Nominatim#Reverse_Geocoding

In [ ]:
if not useGoogleMapsAPI:
    if getSpatialNamesFromOSM:
        osmNominatimURL="http://nominatim.openstreetmap.org/search?format=json&namedetails=1&q="
        # if you re-run this cell you might want to uncomment the following line
        latLng=dict()
        names=dict()
    
        maxItems=df4.shape[0]
        counter=0
    
        printLog("Fetching geolocations from OpenStreetMaps...")
        for row in df4.iterrows():
            counter=counter+1
            if counter%5000==0:
                printLog("\tProcessed %i items of %i"%(counter,maxItems))
            if not row[1]["spatialClusterName"]:
                # in some cases, i.e., when the cluster was corrected manually above, 
                # we will ignore the spatialClean replacement
                loc=str(row[1]["spatialClean"])
                if loc:
                    if not loc.lower() in ignoredSpatialNamesLowerCase:
                        printLog("No spatial cluster name for %s. Taking %s instead."%(row[1]["PPN"],loc))
                    else:
                        loc=None
            else:
                loc=str(row[1]["spatialClusterName"])
            if loc:
                if loc not in latLng:
                #printLog(loc)
                    locURL=osmNominatimURL+urllib.parse.quote(loc).replace(" ","+")
                    try:
                        data = json.load(urllib.request.urlopen(locURL))
                        latLng[loc]=dict()
                        if len(data)>0:
                            #print data[0]
                            latLng[loc][u'lat']=data[0]["lat"]
                            latLng[loc][u'lng']=data[0]["lon"]
                            #print loc
                            names[loc]=dict()
                            if len(data[0]['namedetails'])>0:
                                for k,v in data[0]['namedetails'].items():
                                    names[loc][k]=v
                            else:
                                names[loc][u'name']=loc
                                #print "No namedetails for "+loc
                            #print names[loc]
                        else:
                            latLng[loc]=None
                    except IOError:
                        printLog("\tCould not open: "+locURL)
                        pickleCompress('./picklez/save_names.picklez',names)
                        pickleCompress('./picklez/save_latLng.picklez',latLng)
                    time.sleep(1) # see http://wiki.openstreetmap.org/wiki/Nominatim_usage_policy
        printLog("Number of coordinates: %i"%len(latLng))
        printLog("Done.")
    else:
        printLog("Skipping OpenStreetMap API calls.")
else:
    printLog("Skipping OpenStreetMap API calls.")

In [ ]:
# for manual additions, you can directly pass an URL as the example below shows
#manualLocation="Kehl"
#data = json.load(urllib.urlopen("http://nominatim.openstreetmap.org/search?format=json&namedetails=1&q="+manualLocation))
#loc=manualLocation
#if len(data)>0:
#    latLng[loc][u'lat']=data[0]["lat"]
#    latLng[loc][u'lng']=data[0]["lon"]
#    names[loc]=dict()
#    if len(data[0]['namedetails'])>0:
#        for k,v in data[0]['namedetails'].items():
#            names[loc][k]=v
#    else:
#        names[loc][u'name']=loc
#else:
#    latLng[loc]=None

In [ ]:
# save everything
if not useGoogleMapsAPI:
    if getSpatialNamesFromOSM:
        pickleCompress('./picklez/osm_names.picklez',names)
        pickleCompress('./picklez/osm_latLng.picklez',latLng)

In [ ]:
if useGoogleMapsAPI:
    latLng=pickleDecompress('./picklez/geo_coordinates.picklez')
else:
    latLng=pickleDecompress('./picklez/osm_latLng.picklez')
    osmNames=pickleDecompress('./picklez/osm_names.picklez')

latitude und longitude aus den spatial names generieren

In [ ]:
def getLat(spatialName):
    if spatialName:
        if spatialName in latLng:
            if latLng[spatialName]:
                return latLng[spatialName][u'lat']
            else:
                return np.nan
    else:
        return np.nan

def getLng(spatialName):
    if spatialName:
        if spatialName in latLng:
            if latLng[spatialName]:
                return latLng[spatialName][u'lng']
            else:
                return np.nan
    else:
        return np.nan

df4['latitude']=df4['spatialClusterName'].apply(getLat)
df4['longitude']=df4['spatialClusterName'].apply(getLng)

### lambda-Ausdruck ansprechen (Church aus theoretischer Informatik...)

In [ ]:
# overwrite every "nulled" column (i.e. NULL or NaN) with -1
df4.ix[df4["dateClean"].isnull(), "dateClean"] = -1
df4.ix[df4["dateClean"]=='nan', "dateClean"] = -1

df4["century"]=df4["dateClean"].apply(lambda x: int(int(x)/100)) # uns interessiert nur das jahrhundert
df4.head(20)

In [ ]:
grpCentury=df4.groupby("century")
grpCentury.describe()

In [ ]:
ax=grpCentury.count().plot()
ax.legend(bbox_to_anchor=(1.1, 1.05))

### Text-based Clustering

In [ ]:
printLog("Number of unique title fields %i"%len(df4.title.unique()))

In [ ]:
#title soll geclustert werden -> textCluster
corpus=df4.title.astype(str)

The next cell will take a while to compute depending on your computer and the size of the corpus. To give an example, the following performance could be observed on a MacBook Pro (13-inch, 2018, Four Thunderbolt 3 Ports, 2,7 GHz Intel Core i7, 16 GB RAM):
```
[2019-02-25 11:08:53.568526]	Clustering text with cluster target size 5000
[2019-02-25 11:08:53.569143]		 Preparing tf*idf model
[2019-02-25 11:08:56.634801]		 Number of feature names: 125000
[2019-02-25 11:08:56.635519]		 K-Means clustering
[2019-02-25 11:17:49.427231]	Done.
```

In [ ]:
# in Python 3 you cannot save files over 4 GB, hence we have to run the clustering everytime  :-/
true_k=5000

if sys.version_info.major>=3 or allowComputationallyExpensiveCalculations:
    printLog("Clustering text with cluster target size %i"%true_k)
    printLog("\t Preparing tf*idf model")
    # from an analysis we know that there are 167,715 different words in the corpus
    # to speed up processing, we limit the vocabulary size to the ca. top-75% of the words
    tfidfvectorizer = TfidfVectorizer(min_df=1,max_features=125000) #max_features is used to limit the vocabulary size 
    Xtfidf=tfidfvectorizer.fit_transform(corpus)
    featNames=tfidfvectorizer.get_feature_names()
    printLog("\t Number of feature names: %i"%len(featNames))

    printLog("\t K-Means clustering")
    km=MiniBatchKMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    # fit the k-means algorithm on the data created above
    km.fit(Xtfidf)
    if sys.version_info.major<3:
        pickleCompress('./picklez/textClustersK_'+str(true_k)+'.picklez',km)
else:
    km=pickleDecompress('./picklez/textClustersK_'+str(true_k)+'.picklez')
    printLog("\t Updating Dataframe")
# add the detected clusters as a new column to the original data frame
df4['textCluster']=km.labels_
printLog("Done.")

In [ ]:
if allowComputationallyExpensiveCalculations:
    # group the data by the cluster and describe it
    df4.groupby('textCluster').describe()

In [ ]:
df4=df4.sort_values(by="textCluster")
df4[['PPN','title','textCluster']].tail()

do the same with the creator column

In [ ]:
#creator soll geclustert werden -> creatorCluster
corpus=df4.creator.astype(str)

In [ ]:
printLog("Number of unique creator fields %i"%len(df4.creator.unique()))

```
[2019-02-25 11:18:40.288281]	Clustering text with cluster target size 20000
[2019-02-25 11:18:40.289133]		 Preparing tf*idf model
[2019-02-25 11:18:40.958502]		 Number of feature names: 26575
[2019-02-25 11:18:40.959311]		 K-Means clustering
[2019-02-25 11:26:49.960392]	Done.
```

In [ ]:
true_k=20000

if sys.version_info.major>=3 or allowComputationallyExpensiveCalculations:
    printLog("Clustering text with cluster target size %i"%true_k)
    printLog("\t Preparing tf*idf model")
    tfidfvectorizer = TfidfVectorizer(min_df=1) #max_features setzen?
    Xtfidf=tfidfvectorizer.fit_transform(corpus)
    featNames=tfidfvectorizer.get_feature_names()
    printLog("\t Number of feature names: %i"%len(featNames))

    printLog("\t K-Means clustering")
    km=MiniBatchKMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    # fit the k-means algorithm on the data created above
    km.fit(Xtfidf)
    if sys.version_info.major<3:
        pickleCompress('./picklez/creatorClustersK_'+str(true_k)+'.picklez',km)
else:
    km=pickleDecompress('./picklez/creatorClustersK_'+str(true_k)+'.picklez')

printLog("Done.")

In [ ]:
# add the detected clusters as a new column to the original data frame
printLog("\t Updating Dataframe")
df4['creatorCluster']=km.labels_

df4=df4.sort_values(by="creatorCluster")
df4[['PPN','creator','creatorCluster']].tail()

In [ ]:
imageBasePath="/Users/david/src/__datasets/"
def getImageAvailability(ppn):
    if ppn:
        if os.path.exists(imageBasePath+"firstpages/"+ppn+".jpg"):
            return "firstpages/"+ppn+".jpg"
        else:
            if os.path.exists(imageBasePath+"titlepages/"+ppn+".jpg"):
                return "titlepages/"+ppn+".jpg"
            else:
                return None
    else:
        return None

printLog("Checking image availability...")
df4['titleImage']=df4['PPN'].apply(getImageAvailability)

recordsWithTitleImage=df4[df4.titleImage.notnull()].shape[0]
recordsWithoutTitleImage=df4[df4.titleImage.isnull()].shape[0]
printLog("Records with title image: %i\twithout title image (multi-volumes): %i"%(recordsWithTitleImage,recordsWithoutTitleImage))
printLog("Done.")

In [ ]:
pickleCompress('./picklez/clean_dataframe_with_century.picklez',df4)

In [ ]:
df4.to_csv("cleanedData.csv",sep=';',header=True, index=False, encoding='utf-8')
df4.to_excel("cleanedData.xlsx",header=True, index=False)

## The Analyses Shall Begin!
safe to continue from here if you resume the notebook...

In [ ]:
# if you have run this notebook from top to bottom, you don't have to reload the data
df4=pickleDecompress('./picklez/clean_dataframe_with_century.picklez')
grpCentury=df4.groupby("century")
#df4.shape

In [ ]:
uniqueValues(df4)

### Speeding things up

dauert ca.40 min mit Zugriff innerhalb der Schleife mittels df4[df4.PPN==ppn].iloc[-1]['title'], deshalb look-up, so dauert alles komplett ca. 40 Sekunden.

In [ ]:
printLog("Creating PPN look-up table of %i unique PPNs."%len(df4.PPN.unique()))
ppnLookup=dict()
for row in df4.iterrows():
    ppn=readDate=str(row[1]["PPN"])
    if ppn not in ppnLookup:
        ppnLookup[ppn]=dict()
    # it is redundant but handy for later JSON exports to save the PPN as a value as well
    ppnLookup[ppn]["ppn"]=ppn
    ppnLookup[ppn]["dateClean"]=str(row[1]["dateClean"])
    ppnLookup[ppn]["title"]=str(row[1]["title"])
    ppnLookup[ppn]["creator"]=str(row[1]["creator"])
    
    if not row[1]["spatialClusterName"]:
        if row[1]["spatialClean"]:
            ppnLookup[ppn]["spatialClusterName"]=row[1]["spatialClean"]
        else:
            ppnLookup[ppn]["spatialClusterName"]="None"
    else:
        ppnLookup[ppn]["spatialClusterName"]=row[1]["spatialClusterName"] 
    ppnLookup[ppn]['spatialClean']=row[1]["spatialClean"]
    ppnLookup[ppn]['spatialRaw']=str(row[1]["coverage"])
    ppnLookup[ppn]['mediatype']=str(row[1]["type"])
    ppnLookup[ppn]['subject']=str(row[1]["subject"])
    # daz
    #ppnLookup[ppn]['source']=str(row[1]["source"])
    ppnLookup[ppn]['publisher']=str(row[1]["publisher"])
    #ppnLookup[ppn]['alternative']=str(row[1]["alternative"])
    ppnLookup[ppn]['lat']=str(row[1]["latitude"])
    ppnLookup[ppn]['lng']=str(row[1]["longitude"])
    ppnLookup[ppn]['textCluster']=str(row[1]["textCluster"])
    ppnLookup[ppn]['creatorCluster']=str(row[1]["creatorCluster"])
    ppnLookup[ppn]['titleImage']=str(row[1]["titleImage"])
    
pickleCompress("./picklez/ppnLookup.picklez",ppnLookup)
printLog("Done.")

In [ ]:
#serialize data per PPN; one large file would be more than 80 MB and thus too much for a transmission to a browser
if serializePPNLookup2JSON:
    printLog("Serializing PPN lookup table to JSON in directory:\n\t"+jsonWebDir)
    for k in ppnLookup:
        dump=json.dumps(ppnLookup[k])

        jsonFile = open(jsonWebDir+k+".json", "w")
        jsonFile.write(dump)
        jsonFile.close()
    printLog("Done.")

### A Creation and Its Place of Origin - Graphing the Library World
build a network graph: do publishers and their publication locations resemble superheroes?
verbindung publisher/creator - spatialClusterName; dazu: dateClean, century, title

In [ ]:
def createGraph(consideredDataFrame):
    # for testing purposes it is handy to limit the number of records
    #consideredDataFrame=consideredDataFrame[(consideredDataFrame.Year>=1961) & (consideredDataFrame.Year<1975)]

    # create an empty graph from the nx (networkx) package imported above
    G=nx.Graph()

    rowCount=0
    seenCreators=[]
    seenLocations=[]
    for row in consideredDataFrame.iterrows():
        rowCount=rowCount+1
        #if rowCount%1000==0:
        #    printLog("Processed %i rows..."%rowCount)
        ppn=row[1]["PPN"]
        creator=str(row[1]["publisher"]).upper()
        if creator=="NAN":
            creator=str(row[1]["creator"]).upper()
        if creator=="NAN":
            creator="Unknown creator"
        
        location=row[1]["spatialClusterName"]
        if location=="NAN":
            location="s. l."
        elif not location:
            location="s. l."
        year=row[1]["dateClean"]
        subject=row[1]["subject"]
        
        #if not creator in G.nodes():
        #if not creator in seenCreators:
        
        G.add_node(creator)
        if not creator=="Unknown creator":
            # the name attribute will be helpful for D3.js visualizations
            G.node[creator]['name'] = creator
            G.node[creator]['year'] = year
            G.node[creator]['type'] = "creator"
            G.node[creator]['subject']=subject
            G.node[creator]['century']=row[1]["century"]
            seenCreators.append(creator)
        else:
            G.node[creator]['name'] = creator
            G.node[creator]['year'] = year
            G.node[creator]['type'] = "no_creator"
        
        #if not location in G.nodes():
        #if not location in seenLocations:
       
        # the name attribute will be helpful for D3.js visualizations
        if not location=="s. l.":
            G.add_node(location)
            G.node[location]['name'] = location
            G.node[location]['year'] = year
            G.node[location]['type'] = "location"
            seenLocations.append(location)
            G.add_edge(creator,location)
        else:
            pass
        # if we would like to add the unknown location, uncomment the following lines and the comment the last line
            #G.add_node(location)
            #G.node[location]['name'] = location
            #G.node[location]['year'] = year
            #G.node[location]['type'] = "no_location"
    
        #print location
        #print creator
        #print "* * *"
    return G

In [ ]:
printLog("Creating graphs and additional data...")

graphsPerCentury=dict()
ppnPerCentury=OrderedDict()
for century in range(7,21): # we know that there are only media from the 7th century on
    if century in grpCentury.groups:
        centDF=grpCentury.get_group(century)
        returnedGraph=createGraph(centDF)
        graphsPerCentury[century]=returnedGraph
        printLog("Graph for century %i built with %i nodes and %i edges."%(century, len(returnedGraph.nodes()),len(returnedGraph.edges())))
        # export as GraphML, which can be read by Gephi
        nx.write_gml(returnedGraph,"graphs/century_"+str(century)+".gml")
        
        # now save the PPNs
        ppnPerCentury[century]=[]
        for row in centDF.iterrows():
            ppn=row[1]["PPN"]
            ppnPerCentury[century].append(ppn)

# create the full graph
returnedGraph=createGraph(df4)
printLog("Graph for all centuries built with %i nodes and %i edges."%(len(returnedGraph.nodes()),len(returnedGraph.edges())))
nx.write_gml(returnedGraph,"graphs/all_centuries.gml")
printLog("Done.")

In [ ]:
def createFullGraph(consideredDataFrame):
    # for testing purposes it is handy to limit the number of records
    #consideredDataFrame=consideredDataFrame[(consideredDataFrame.Year>=1961) & (consideredDataFrame.Year<1975)]

    # create an empty graph from the nx (networkx) package imported above
    G=nx.Graph()

    rowCount=0
    seenCreators=[]
    seenLocations=[]
    for row in consideredDataFrame.iterrows():
        rowCount=rowCount+1
        #if rowCount%1000==0:
        #    printLog("Processed %i rows..."%rowCount)
        ppn=row[1]["PPN"]
        creator=str(row[1]["publisher"]).upper()
        if creator=="NAN":
            creator=str(row[1]["creator"]).upper()
        if creator=="NAN":
            creator="Unknown creator"
        
        location=row[1]["spatialClusterName"]
        if location=="NAN":
            location="s. l."
        elif not location:
            location="s. l."
        year=row[1]["dateClean"]
        subject=row[1]["subject"]
        
        #if not creator in G.nodes():
        #if not creator in seenCreators:
        
        G.add_node(creator)
        if not creator=="Unknown creator":
            # the name attribute will be helpful for D3.js visualizations
            G.node[creator]['name'] = creator
            G.node[creator]['year'] = year
            G.node[creator]['type'] = "creator"
            G.node[creator]['subject']=subject
            G.node[creator]['century']=row[1]["century"]
            seenCreators.append(creator)
        else:
            G.node[creator]['name'] = creator
            G.node[creator]['year'] = year
            G.node[creator]['type'] = "no_creator"
        
        #if not location in G.nodes():
        #if not location in seenLocations:
       
        # the name attribute will be helpful for D3.js visualizations
        if not location=="s. l.":
            G.add_node(location)
            G.node[location]['name'] = location
            G.node[location]['year'] = year
            G.node[location]['type'] = "location"
            seenLocations.append(location)
            G.add_edge(creator,location)
        else:
            pass
        # if we would like to add the unknown location, uncomment the following lines and the comment the last line
            #G.add_node(location)
            #G.node[location]['name'] = location
            #G.node[location]['year'] = year
            #G.node[location]['type'] = "no_location"
    
        #print location
        #print creator
        #print "* * *"
        
        G.add_node(ppn)
        G.node[ppn]['name'] = str(ppn)
        G.node[ppn]['year'] = year
        G.node[ppn]['type'] = "PPN"
        G.node[ppn]['century']=row[1]["century"]
        G.add_edge(ppn,creator)

    return G

In [ ]:
printLog("Creating graphs incl. PPNs...")

graphsPerCentury=dict()
for century in range(7,21): # we know that there are only media from the 7th century on
    if century in grpCentury.groups:
        centDF=grpCentury.get_group(century)
        returnedGraph=createFullGraph(centDF)
        graphsPerCentury[century]=returnedGraph
        gmlPath="graphs/century_ppn_"+str(century)+".gml"
        # export as GraphML, which can be read by Gephi
        nx.write_gml(returnedGraph,gmlPath)
        printLog("Graph for century %i built with %i nodes and %i edges (see %s)."%(century, len(returnedGraph.nodes()),len(returnedGraph.edges()),gmlPath))
        

# create the full graph
returnedGraph=createFullGraph(df4)
gmlPath="graphs/all_centuries_ppn.gml"
nx.write_gml(returnedGraph,gmlPath)
printLog("Graph for all centuries built with %i nodes and %i edges (see %s)."%(len(returnedGraph.nodes()),len(returnedGraph.edges()),gmlPath))
printLog("Done.")

In [ ]:
# takes ca. 45 min for a graph built with 16368 nodes and 18539 edges on my MacBook Pro, i.e., with the full data set
# with http://networkx.github.io/documentation/latest/reference/generated/networkx.drawing.layout.spring_layout.html#networkx.drawing.layout.spring_layout
# theory behind it: https://en.wikipedia.org/wiki/Force-directed_graph_drawing
#printLog("Preparing to draw...")

#nx.draw(G)
#plt.savefig("graph.pdf")
#printLog("Done.")

generic graph building

In [ ]:
def createGenericGraph(consideredDataFrame):
    # for testing purposes it is handy to limit the number of records
    #consideredDataFrame=consideredDataFrame[(consideredDataFrame.Year>=1961) & (consideredDataFrame.Year<1975)]

    # create an empty graph from the nx (networkx) package imported above
    G=nx.Graph()

    rowCount=0
    seenCreators=[]
    seenLocations=[]
    # [u'PPN', u'alternative', u'creator', u'dataProvider', u'date', 
    # u'description', u'format', u'identifier', u'isShownAt', u'issued', 
    # u'object', u'provider', u'publisher', u'relation', u'rights', u'source', 
    # u'spatial', u'subject', u'title', u'type', u'spatialClean', u'dateClean', 
    # u'spatialCluster', u'spatialClusterName', u'latitude', u'longitude', u'century']
    ignoreList=[u'rights', u'issued',u'source',u'spatial',u'dataProvider', u'date',u'format',u'spatialCluster',u'century', u'provider']
    for row in consideredDataFrame.iterrows():#itertuples(): 
        ppn=val=row[1]["PPN"]
        G.add_node(ppn)
        G.node[ppn]['name'] = ppn
        G.node[ppn]['type'] = "PPN"
            
        keys=row[1].keys()
        for k in keys:
            elementEmpty=False
            if k not in ignoreList:
                #print k
                val=row[1][k]
                if val: 
                    if type(val) is float:
                        if not math.isnan(val):
                            #print row[1][k]
                            pass
                        else:
                            #print "I DON'T KNOW!"
                            elementEmpty=True
                    else:
                        #print row[1][k]
                        pass
                else:
                    #print "I DON'T KNOW!"
                    elementEmpty=True
                
                # only process non-empty elements
                if not elementEmpty:
                    if not val in G.nodes():
                        G.add_node(val)
                        G.node[val]['name'] = val
                        G.node[val]['type'] = k
                    
                    G.add_edge(ppn,val)
            
        # spatialClean auf spatialClusterName binden, den rest dann mit spatialClusterName
        # spatialClusterName erhält als Attribut latitude/longitude

       

    return G

In [ ]:
if allowComputationallyExpensiveCalculations:
    saveDir="./graphs/generics/"
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
        
    printLog("Creating graphs incl. PPNs...")

    graphsPerCentury=dict()
    for century in range(7,21):#(7,21): # we know that there are only media from the 7th century on
        if century in grpCentury.groups:
            centDF=grpCentury.get_group(century)
            returnedGraph=createGenericGraph(centDF)
            graphsPerCentury[century]=returnedGraph
            gmlPath="graphs/generics/full_"+str(century)+".gml"
            # export as GraphML, which can be read by Gephi
            nx.write_gml(returnedGraph,gmlPath)
            printLog("Graph for century %i built with %i nodes and %i edges (see %s)."%(century, len(returnedGraph.nodes()),len(returnedGraph.edges()),gmlPath))
    printLog("Done.")

# TO DO
Graph mit allen Städten und PPNs eines Spatial Clusters

xxx

In [ ]:
grpCentury.get_group(17).count()

### Exploring the Collection by Visual Content
* reading the feature files takes approx. 12 min
* erklären, woher die CBIR features kommen (https://en.wikipedia.org/wiki/Bag-of-words_model_in_computer_vision)

![Visual Words](img/visword.jpg)
By Masterwaw - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=19645418

Scale-invariant feature transform, reference for local feature detection

![Visual Word Generation](img/visword_generation2.png)

int sampleDocumentsToCreateCodebook = 5000; 
int numberOfClusters = 1000;

next step takes ca. 40 minutes if the raw features are loaded from disk and converted

In [ ]:
featureBaseDir="./featureFiles.5k1k/"
missingPPNs=[]
readPPNs=[]
featuresPPN=[]
featsPerCentury=dict()
readPpnPerCentury=dict()

printLog("Loading features...")
if reinterpretVisualWordRawFeatures:
    for century in range(7,21):
        if century in grpCentury.groups:
            featsPerCentury[century]=[]
    for century in range(7,21):
        if century in grpCentury.groups:
            readPpnPerCentury[century]=[]

    index=0
    for row in df4.iterrows():
        index=index+1
        if index%10000==0:
            printLog("Processed %i documents."%index)
        ppn=str(row[1]["PPN"])
        if os.path.isfile(featureBaseDir+ppn+".csv"):
            #print ppn+" okay."
            featFile=open(featureBaseDir+ppn+".csv")
            for line in featFile:
                feature=line
            tokens=feature.split()
            harray=[]
            for t in tokens:
                harray.append(int(t,16))
            featFile.close()

            readPPNs.append(ppn)
            featuresPPN.append(np.array(harray,dtype=np.uint8))
            # check to which century the feature belongs
            for century in range(7,21):
                if century in grpCentury.groups:
                    if ppn in ppnPerCentury[century]:
                        readPpnPerCentury[century].append(ppn)
                        featsPerCentury[century].append(np.array(harray,dtype=np.uint8))
        else:
            missingPPNs.append(ppn)
    printLog("Done.")
    printLog("Number of missing PPNs: %i"%len(missingPPNs))
    
    # pickling takes about 15 minutes
    pickleCompress('./picklez/missingPPNs.picklez',missingPPNs)
    pickleCompress('./picklez/readPPNs.picklez',readPPNs)
    pickleCompress('./picklez/featuresPPN.picklez',featuresPPN)
    pickleCompress('./picklez/featsPerCentury.picklez',featsPerCentury)
    pickleCompress('./picklez/readPpnPerCentury.picklez',readPpnPerCentury)
    printLog("Pickling completed.")
else:
    # takes about 1 minute
    missingPPNs=pickleDecompress('./picklez/missingPPNs.picklez')
    readPPNs=pickleDecompress('./picklez/readPPNs.picklez')
    featuresPPN=pickleDecompress('./picklez/featuresPPN.picklez')
    featsPerCentury=pickleDecompress('./picklez/featsPerCentury.picklez')
    readPpnPerCentury=pickleDecompress('./picklez/readPpnPerCentury.picklez')
    printLog("Loading from disk completed.")

* Clustering of 101031 elements started with 1000 as cluster target size: 3 min
* note that the cluster labels vary for each run

In [ ]:
# working with all features will most likely halt your computer because of the memory consumption if you use KMeans!
feats=featuresPPN#[:20000] 

# define the number of clusters to be found
true_k=1000
printLog("Clustering of %i elements started with %i as cluster target size."%(len(feats),true_k))
# initialize the k-means algorithm
#km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# we will rely on the mini batch k-means algorithm due to performance consideration otherwise your computer might crash...
km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

# apply the algorithm on the data
km.fit(feats)
printLog("Clustering finished.")
km.labels_

In [ ]:
saveDir="./html/_clusteroverview_allcents/"
if not os.path.exists(saveDir):
        os.makedirs(saveDir)
        
imgDir="../../web/thumbnails/"
#imgDir="file:///Volumes/2TB_WD/sbb_images/tmp/"
htmlHead="<html><head></head><body bgcolor='#000000'>"
htmlTail="</body></html>"
clusters=dict()
for i,val in enumerate(km.labels_):
    if val not in clusters:
        clusters[val]=[]
    clusters[val].append(readPPNs[i])
#print clusters
for i in clusters:
    htmlOut=open(saveDir+str(i)+".html","w")
    htmlOut.write(htmlHead+"\n")
    htmlOut.write("<a href='"+str(i-1)+".html'>last</a> &nbsp;"+"<a href='"+str(i+1)+".html'>next</a>\n"+"<br />")
    for ppn in clusters[i]:
        htmlOut.write("<img width='170' src='"+imgDir+ppn+".jpg' />\n")
    htmlOut.write(htmlTail)
    htmlOut.close()

repeat the same step for each century...

In [ ]:
clustersPerCentury=dict()
maxClusterAmount=100
printLog("Starting clustering per century...")
for century in featsPerCentury:
    maxClusters=len(featsPerCentury[century])
    # define the number of clusters to be found
    true_k=int(maxClusters*0.1+1)
    # restrict the number of clusters to prevent extremely large clusters
    if true_k>(maxClusterAmount*2):
        true_k=maxClusterAmount+int(maxClusters*0.005)
    elif true_k>maxClusterAmount:
        true_k=maxClusterAmount+int(maxClusters*0.01)
    printLog("Clustering of %i element(s) started with %i as cluster target size for century %i."%(maxClusters,true_k,century))
    # initialize the k-means algorithm
    #km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    # we will rely on the mini batch k-means algorithm due to performance consideration otherwise your computer might crash...
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

    # apply the algorithm on the data
    km.fit(featsPerCentury[century])
    clustersPerCentury[century]=km.labels_
printLog("Clustering finished.")

Computing 'centroids' takes up to 6 minutes;

In [ ]:
clusterCentroidsPerCentury=dict()
numberCentroids=0

printLog("Computing 'centroids' for...")

for century in clustersPerCentury:
    printLog("\tcentury %i"%century)
    clusters=dict()
    centFeats=dict()

    for i,val in enumerate(clustersPerCentury[century]):
        if val not in centFeats:
            centFeats[val]=[]
        if val not in clusters:
            clusters[val]=[]
        index=readPPNs.index(readPpnPerCentury[century][i])
        clusters[val].append(readPpnPerCentury[century][i])
        centFeats[val].append(featuresPPN[index])

    clusterCentroidsPerCentury[century]=dict()
    for cluster in centFeats:
        r=centFeats[cluster]
        meanDistances=[]
        D=pairwise_distances(r,r)
        #A distance matrix D such that D_{i, j} is the distance between the ith and jth vectors of the given matrix X, if Y is None.
        #If Y is not None, then D_{i, j} is the distance between the ith array from X and the jth array from Y.
        # distance between element 0 and 13 (=0.0 if X and Y are anti-correlated)
        #D[0][13]
        for row in D:
            # each row in D stands for one document and its distances to all other documents
            # by calculating its mean, we compute how dissimilar this document is to all others
            meanDistances.append(np.mean(row))
        #print meanDistances
        minVal=np.min(meanDistances)
        index=meanDistances.index(minVal)
        clusterCentroidsPerCentury[century][cluster]=clusters[cluster][index]
        numberCentroids=numberCentroids+1
        #print str(cluster)+": "+str(clusters[cluster][index])+" (of %i elements)"%len(meanDistances)

printLog("Done computing %i 'centroids'."%numberCentroids)


* iterate over all centuries and save output per centroids
* CSV output is for the visualization with the web-based QA tools

In [ ]:
saveDir="./html/_clusteroverview_per_century/"
if not os.path.exists(saveDir):
        os.makedirs(saveDir)
        
printLog("Creating HTML and CSV output...")

# limits the shown PPNs per century-separated cluster
limitClusterCentroidsPerCentury=10
# for the cluster detail views limitClusterCentroidsPerCentury*centuryLimitFactor elements will be displayed per centroid
centuryLimitFactor=3

csvOut=open("./web/data/clusters.csv","w")
csvOut.write("id,value\n")
rootNode="all."
csvOut.write("all,"+"\n")

for century in clustersPerCentury:
    largestClusterSize=0
    largestCluster=None
    
    csvCenturyOut=open("./web/data/"+str(century)+".csv","w")
    csvCenturyOut.write("id,value\n")
    rootCenturyNode="all"
    csvCenturyOut.write(rootCenturyNode+","+"\n")
    
    csvOut.write("all."+str(century)+","+"\n")
    
    saveDir="./html/_clusteroverview_per_century/"+str(century)+"/"
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    imgDir="../../../web/thumbnails/"
    #imgDir="file:///Volumes/2TB_WD/sbb_images/tmp/"
    htmlHead="<html><head></head><body bgcolor='#000000'>"
    htmlTail="</body></html>"
    
    clusters=dict()
    for i,val in enumerate(clustersPerCentury[century]):
        if val not in clusters:
            clusters[val]=[]
        clusters[val].append(readPpnPerCentury[century][i])
        

    clusterSizes=[]
    shownCentroidCount=0
    shownMoreCentroidsAvailable=False
    
    noClustersInCentury=len(clusters)
    for i in clusters:
        clusterSizes.append(len(clusters[i]))
        
        if largestClusterSize<len(clusters[i]):
            largestClusterSize=len(clusters[i])
            largestCluster=i
        
        # the cluster's centroid
        # 14/14: PPN789774356
        #<br/>
        #<img src='file:///Users/david/Documents/src/python/CulturalAnalytics/tmp/PPN789774356.jpg' />
        #print "\tCentroid for cluster "+str(i)+": "+str(clusterCentroidsPerCentury[century][i])
        centroid="<img src='"+imgDir+str(clusterCentroidsPerCentury[century][i])+".jpg' />\n"+"<br/>\n"
    
        if shownCentroidCount<limitClusterCentroidsPerCentury:
            shownCentroidCount=shownCentroidCount+1
            csvOut.write(rootNode+str(century)+"."+str(clusterCentroidsPerCentury[century][i])+",\n")
        else:
            if not shownMoreCentroidsAvailable:
                csvOut.write(rootNode+str(century)+".more,\n")
                shownMoreCentroidsAvailable=True
                
        csvCenturyOut.write(rootCenturyNode+"."+str(clusterCentroidsPerCentury[century][i])+",\n")
        
        
        
        htmlOut=open(saveDir+str(i)+".html","w")
        htmlOut.write(htmlHead+"\n")
        #htmlOut.write("<a href='"+str(century)+str(i-1)+".html'>last</a> &nbsp;"+"<a href='"+str(century)+str(i+1)+".html'>next</a>\n"+"<br />")
        htmlOut.write("<a href='"+str(i-1)+".html'>last</a> &nbsp;"+"<a href='"+str(i+1)+".html'>next</a>\n"+"<br />")
        
        htmlOut.write(centroid)
        
        centroidPPN=str(clusterCentroidsPerCentury[century][i])#str(clusters[i][0])
        csvPPNOut=open("./web/data/"+centroidPPN+".csv","w")
        csvPPNOut.write("id,value\n")
        rootPPNNode="all"
        csvPPNOut.write(rootPPNNode+","+"\n")
        csvPPNOut.write(rootPPNNode+"."+centroidPPN+",\n")
        ppnCount=0
    
        for ppn in clusters[i]:
            htmlOut.write("<img width='170' src='"+imgDir+ppn+".jpg' alt='"+ppn+"'/>\n")
            #csvOut.write(rootNode+str(century)+"."+str(clusterCentroidsPerCentury[century][i])+"."+ppn+",\n")
            if ppnCount<limitClusterCentroidsPerCentury*centuryLimitFactor:
                ppnCount=ppnCount+1
                csvCenturyOut.write(rootCenturyNode+"."+str(clusterCentroidsPerCentury[century][i])+"."+ppn+",\n")
            else:
                csvCenturyOut.write(rootCenturyNode+"."+str(clusterCentroidsPerCentury[century][i])+".more,\n")
                break
            
            csvPPNOut.write(rootPPNNode+"."+centroidPPN+"."+ppn+",\n")
        csvPPNOut.close()
        htmlOut.write(htmlTail)
        htmlOut.close()
    
    csvCenturyOut.close()
    print("\tLargest cluster for century %i is %i with %i elements."%(century,largestCluster,largestClusterSize))
    print("\t\tNumber of clusters: %i"%noClustersInCentury)
    print("\t\tMean cluster size: %s"%str(np.mean(clusterSizes)))
    print("\t\tCluster size standard deviation: %s"%str(np.std(clusterSizes)))
    print("\t\tMin. cluster size: %s"%str(np.amin(clusterSizes)))
    print("\t\tMax. cluster size: %s"%str(np.amax(clusterSizes)))
csvOut.close()
printLog("Done.")

### A Visual Timeline of Publications
plotting of the "centroids"

In [ ]:
imgDir="../web/thumbnails/"
htmlHead="<html><head></head><body bgcolor='#000000'>"
htmlTail="</body></html>"
centroidPath="html/_centroids.html"

printLog("Saving centroid overview HTML page at: "+centroidPath)
htmlOut=open(centroidPath,"w")
htmlOut.write(htmlHead)
for century in clustersPerCentury:
    htmlOut.write("<h1 style='color:white;'>"+str(century)+"</h1>\n")
    for centroid in clusterCentroidsPerCentury[century]:
        htmlOut.write("<img width='170' src='"+imgDir+clusterCentroidsPerCentury[century][centroid]+".jpg' />\n")
htmlOut.write(htmlTail)
htmlOut.close()
printLog("Done.")

we can also extend the idea and create a graph of the data
to base a nice visualization of the clusters...

In [ ]:
imgDir="./web/thumbnails/"
printLog("Creating overview graph...")
G=nx.Graph()

lastCentury="7"
for century in clustersPerCentury: 
#for century in [7,10,11,12,13,14,15,16]:#range(12,15):
    strCentury=str(century)
    G.add_node(strCentury)
    G.node[strCentury]['name'] = strCentury
    G.node[strCentury]['type'] = "century"
    
    for centroid in clusterCentroidsPerCentury[century]:
        ppn=str(clusterCentroidsPerCentury[century][centroid])
        imagePath=imgDir.replace("file://","")+ppn+".jpg"
        G.add_node(ppn)
        G.node[ppn]['name'] = ppn
        if os.path.isfile(imagePath):
            G.node[ppn]['name'] = ppn
            G.node[ppn]['imagePath'] = ppn
        else:
            G.node[ppn]['name'] = ppn
            G.node[ppn]['imagePath'] = "none"
        G.node[ppn]['title'] = ppnLookup[ppn]['title']
        G.node[ppn]['creator'] = ppnLookup[ppn]['creator']
        if ppnLookup[ppn]['spatialClusterName']:
            G.node[ppn]['location'] = ppnLookup[ppn]['spatialClusterName']
        G.node[ppn]['locationRaw'] =ppnLookup[ppn]['spatialRaw']
        
        G.node[ppn]['mediatype'] =ppnLookup[ppn]['mediatype']
        G.node[ppn]['subject'] =ppnLookup[ppn]['subject']
        G.node[ppn]['source'] =ppnLookup[ppn]['source']
        G.node[ppn]['publisher'] =ppnLookup[ppn]['publisher']
        G.node[ppn]['alternative'] =ppnLookup[ppn]['alternative']
            
        G.node[ppn]["century"]=century
        G.node[ppn]['dateClean'] =ppnLookup[ppn]['dateClean']
        G.node[ppn]["cluster"]=str(centroid)
        G.node[ppn]['lat']=ppnLookup[ppn]['lat']
        G.node[ppn]['lng']=ppnLookup[ppn]['lng']
        G.node[ppn]['type'] = "image"
        G.node[ppn]['textCluster'] =ppnLookup[ppn]['textCluster']
        G.node[ppn]['creatorCluster'] =ppnLookup[ppn]['creatorCluster']
        
        G.add_edge(strCentury,ppn)
    G.add_edge(strCentury,lastCentury)
    lastCentury=strCentury
    
nx.write_gml(G,"graphs/century.gml")
d = json_graph.node_link_data(G)
jsonPath='./web/data/century.json'
#jsonPath='./force/century_test.json'
saveDir="./web/force/"
if not os.path.exists(saveDir):
        os.makedirs(saveDir)
        
json.dump(d, open(jsonPath,'w'))
printLog("Done (see %s)."%jsonPath)

In [ ]:
printLog("Creating cluster graph output...")
imgDir="./web/thumbnails/"

saveDir="./web/data/clusters/"
if not os.path.exists(saveDir):
        os.makedirs(saveDir)

for century in clustersPerCentury:
    printLog("Processing century "+str(century))
    saveDir="./web/data/clusters/"+str(century)+"/"
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)

    clusters=dict()
    for i,val in enumerate(clustersPerCentury[century]):
        if val not in clusters:
            clusters[val]=[]
        clusters[val].append(readPpnPerCentury[century][i])

    for i in clusters:
        G=nx.Graph()
        # the cluster's centroid
        centroid=str(clusterCentroidsPerCentury[century][i])
        G.add_node(centroid)
        G.node[centroid]['type'] = "centroid"
        
        for ppn in clusters[i]:
            imagePath=imgDir.replace("file://","")+ppn+".jpg"
            #dateClean=str(df4[df4.PPN==ppn].iloc[-1]['dateClean'])
            dateClean=ppnLookup[ppn]["dateClean"]
            G.add_node(dateClean)
            G.node[dateClean]['name'] = dateClean
            G.node[dateClean]['type'] = "dateClean"
            G.add_edge(centroid,dateClean)
            
            G.add_node(ppn)
            G.node[ppn]['name'] = ppn
            if os.path.isfile(imagePath):
                G.node[ppn]['name'] = ppn
                G.node[ppn]['imagePath'] = ppn
            else:
                G.node[ppn]['name'] = ppn
                G.node[ppn]['imagePath'] = "none"
            #G.node[ppn]['title'] = df4[df4.PPN==ppn].iloc[-1]['title']
            #G.node[ppn]['creator'] = str(df4[df4.PPN==ppn].iloc[-1]['creator'])
            #G.node[ppn]['location'] = df4[df4.PPN==ppn].iloc[-1]['spatialClusterName']
            G.node[ppn]['title'] = ppnLookup[ppn]["title"]
            G.node[ppn]['creator'] = ppnLookup[ppn]["creator"]
            if ppnLookup[ppn]["spatialClusterName"]:
                G.node[ppn]['location'] = ppnLookup[ppn]["spatialClusterName"]
            else:
                G.node[ppn]['location']="none"
            G.node[ppn]['locationRaw'] =ppnLookup[ppn]['spatialRaw']
            
            G.node[ppn]['mediatype'] =ppnLookup[ppn]['mediatype']
            G.node[ppn]['subject'] =ppnLookup[ppn]['subject']
            G.node[ppn]['source'] =ppnLookup[ppn]['source']
            G.node[ppn]['publisher'] =ppnLookup[ppn]['publisher']
            G.node[ppn]['alternative'] =ppnLookup[ppn]['alternative']
            G.node[ppn]['dateClean'] =ppnLookup[ppn]['dateClean']
            G.node[ppn]['lat']=ppnLookup[ppn]['lat']
            G.node[ppn]['lng']=ppnLookup[ppn]['lng']
            
            if ppn==centroid:
                G.node[centroid]['type'] = "centroid"
            else:
                G.node[ppn]['type'] = "image"
            
            G.add_edge(dateClean,ppn)
            
        nx.write_gml(G,saveDir+str(i)+".gml")
        d = json_graph.node_link_data(G)
        jsonPath='./web/data/clusters/'+str(century)+'/'+str(i)+'.json'
        json.dump(d, open(jsonPath,'w'))
printLog("Done.")

## Geospatial Extravaganza

In [ ]:
# points are latitude, longitude
# Latitudes range from -90 to 90.
# Longitudes range from -180 to 180
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
test=(-95,-161)
x=vincenty(newport_ri, test)
print(x.kilometers)

In [ ]:
index=0
ppnPosition=[]
ppnPositionLabels=[]
ppnPositionSpatialClean=[]

printLog("Extracting latitude and longitude...")

for row in df4.iterrows():
    index=index+1
    if index%10000==0:
        printLog("Processed %i documents."%index)
    ppn=str(row[1]["PPN"])
    try:
        lat=float(row[1]["latitude"])
        lng=float(row[1]["longitude"])
        spatialClean=row[1]["spatialClean"]
        if math.isnan(lat) and math.isnan(lng):
            pass
        else:
            ppnPositionLabels.append(ppn)
            ppnPositionSpatialClean.append(spatialClean)
            ppnPosition.append((lat,lng))
    except TypeError:
        #print(row[1]["latitude"])
        pass
    
printLog("Found %i PPNs with coordinates." %len(ppnPositionLabels))

In [ ]:
plt.title('Distribution of Geo-Spatial Coordinates')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.scatter(*zip(*ppnPosition),alpha=0.1)
if saveFiguresAsPDF:
    plt.savefig('./figures/sample.pdf')

* Latitudes range from -90 to 90. -> x-Achse sollte 180 breit sein, Mitte ist 90
* Longitudes range from -180 to 180 -> y-Achse sollte 360 breit sein, Mitte ist 180


In [ ]:
feats=ppnPosition

# define the number of clusters to be found
true_k=30
printLog("Clustering of %i elements started with %i as cluster target size."%(len(feats),true_k))
# initialize the k-means algorithm
#km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# we will rely on the mini batch k-means algorithm due to performance consideration otherwise your computer might crash...
km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

# apply the algorithm on the data
km.fit(feats)
printLog("Clustering finished.")
km.labels_

bounding boxes aus http://boundingbox.klokantech.com/

Visualization with GeoJSON (http://geojson.org/) https://pypi.python.org/pypi/geojson and OpenLayers  http://openlayers.org/en/latest/doc/ http://openlayers.org/en/latest/apidoc/




In [ ]:
regions=["Europe","Africa","Asia","Australia","SouthAmerica","NorthAmerica"]
regionBoundingBox=dict()
#westlimit=-22.5; southlimit=33.6; eastlimit=58.4; northlimit=82.9
regionBoundingBox["Europe"]=[33.6,82.9,-22.5,58.4]
#africa westlimit=-22.9; southlimit=-63.7; eastlimit=58.0; northlimit=37.2
regionBoundingBox["Africa"]=[-63.7,37.2,-22.9,58.0]
#asia westlimit=29.5; southlimit=-11.4; eastlimit=-168.4; northlimit=81.2
regionBoundingBox["Asia"]=[-11.4,81.2,-168.4,29.5]
#australia westlimit=112.5; southlimit=-50.4; eastlimit=-162.9; northlimit=-10.0
regionBoundingBox["Australia"]=[-50.4,-10.0,-162.9,112.5]
#south america westlimit=-119.5; southlimit=-57.0; eastlimit=-29.9; northlimit=28.1
regionBoundingBox["SouthAmerica"]=[-57.0,28.1,-119.5,-29.9]
#north america westlimit=-169.1; southlimit=23.7; eastlimit=-20.0; northlimit=71.4
regionBoundingBox["NorthAmerica"]=[23.7,71.4,-169.1,-20.0]


# man muss checken in welchen intervallen die kontinente liegen, insb. bei 2 und 3
def getRegion(lat,lng):
    for region in regions:
        if region in regionBoundingBox:
            if regionBoundingBox[region][0] <= lat <= regionBoundingBox[region][1]:
                if regionBoundingBox[region][2] <= lng <= regionBoundingBox[region][3]:
                    return region

    return "other"

#print getRegion(51,10)



In [ ]:
# combine all arrays and check if there is more than one location associated with a PPN
# ppnPosition is (lat,lng)
printLog("Creating GeoJSON data...")
multipleCitiesPPN=pickleDecompress("./picklez/multipleSpatialNamesPerPPN.picklez")

baseDir="./web/data/layers/"
fileName=baseDir+"test.json"
locationFeatures=dict()
locationFeaturesGlobal=[]

spots=zip(ppnPositionLabels, ppnPositionSpatialClean,ppnPosition)

for spot in spots:
    ppn=spot[0]
    leadingLoc=spot[1]
    latLng=spot[2]
    # GeoJSON points are in longitude , latitude but our storage is lat/lng
    my_point = gj.Point((latLng[1], latLng[0]))

    region=getRegion(latLng[0],latLng[1])
    if region not in locationFeatures:
        locationFeatures[region]=[]
        print("Adding "+region)
    else:
        locationFeatures[region].append(gj.Feature(geometry=my_point, properties={"title": leadingLoc+" "+ppn, "ppn":ppn}))
    
    locationFeaturesGlobal.append(gj.Feature(geometry=my_point, properties={"title": leadingLoc+" "+ppn, "ppn":ppn}))
    #print ppn
    if ppn in multipleCitiesPPN:
        array=multipleCitiesPPN[ppn]
        #if there are alternative locations for this PPN
        if len(array)>1:
            #print leadingLoc+" "+ppn
            #print "\t"+"; ".join(array).encode("utf-8")
            pass

for region in locationFeatures:
    geoJSON_collection=gj.FeatureCollection(locationFeatures[region])
    dump = gj.dumps(geoJSON_collection, sort_keys=True)

    jsonFile = open(baseDir+region.lower()+".json", "w")
    jsonFile.write(dump)
    jsonFile.close()
    

geoJSON_collection=gj.FeatureCollection(locationFeaturesGlobal)
dump = gj.dumps(geoJSON_collection, sort_keys=True)

globalJSONPath=baseDir+"global.json"
jsonFile = open(globalJSONPath, "w")
jsonFile.write(dump)
jsonFile.close()
printLog("Serialized %i metadata records.\n\tSaved global JSON document at %s." %(len(locationFeaturesGlobal),globalJSONPath))

printLog("Done.")

In [ ]:
#print spots.index("PPN766441857")
#print(spots[10])
#print(getRegion(spots[10][2][0],spots[10][2][1]))

In [ ]:
# we will create an inverted mapping from alternative location names to "leading" location names
# the resulting dicts "main" key will be the leading character of the alternative location name 
# within this map will be a mapping to the "leading" location name, e.g., 'Corfu' would be mapped to 'Korfu'
# lastLetter=r[0].lower()
osmNames=pickleDecompress('./picklez/osm_names.picklez')
osmAlternativesSorted=dict()
for leadLoc in osmNames:
    #print leadLoc
    for v in osmNames[leadLoc].values():
        leadingLetter=v[0].lower()
        if not leadingLetter in osmAlternativesSorted:
            osmAlternativesSorted[leadingLetter]=dict()
        else:
            osmAlternativesSorted[leadingLetter][v]=leadLoc

In [ ]:
# osmNames(key): alle alternativen titel in der Form names[loc][u'name']:
# u'Ems': {u'name:nl': u'Eems', u'name': u'Ems', u'name:de': u'Ems', u'name:la': u'Amisia'...
# latLng(key)[lat|lng]: latitude/longitude pro key

# Ideen

* Timeline und Grafisches Aussehen, x-Achse: Zeit, y-Achse. Farbe? Brightness? Entropy? Abweichung vom Referenzbild (Distanz zum QBE)? https://www.slideshare.net/formalist/how-and-why-study-big-cultural-data-v2-15552598 #43
* Dominante Farbe bestimmen, als 3D-Punkt nehmen und schauen, ob es mit etwas korreliert?
* Stabi-URL:



In [ ]:
#g=df4.groupby("publisher")
#g.describe()

In [ ]:
#g.count().plot(legend=False)

# Service Functions
to avoid problems with JSON access you should load the web pages from your own HTTP server

In [ ]:
if True:#launchHTTPServer:
    # the resulting HTTP service will listen on port 8000 and open the main page in the browser
    import http_server
    http_server.load_url('web/webapps/index.html')